# Google Spanner

> [Spanner](https://cloud.google.com/spanner) is a highly scalable database that combines unlimited scalability with relational semantics, such as secondary indexes, strong consistency, schemas, and SQL providing 99.999% availability in one easy solution.

This notebook goes over how to use `Spanner Graph` for GraphRAG with the custom retriever `SpannerGraphVectorContextRetriever` and compares the response of GraphRAG with conventional RAG.

Learn more about the package on [GitHub](https://github.com/googleapis/langchain-google-spanner-python/).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/langchain-google-spanner-python/blob/main/docs/graph_rag.ipynb)

## Before You Begin

To run this notebook, you will need to do the following:

 * [Create a Google Cloud Project](https://developers.google.com/workspace/guides/create-project)
 * [Enable the Cloud Spanner API](https://console.cloud.google.com/flows/enableapi?apiid=spanner.googleapis.com)
 * [Create a Spanner instance](https://cloud.google.com/spanner/docs/create-manage-instances)
 * [Create a Spanner database](https://cloud.google.com/spanner/docs/create-manage-databases)

### 🦜🔗 Library Installation
The integration lives in its own `langchain-google-spanner` package, so we need to install it.

In [ ]:
%pip install --upgrade --quiet langchain-google-spanner langchain-google-vertexai langchain-experimental spanner-graph-notebook

**Colab only:** Uncomment the following cell to restart the kernel or use the button to restart the kernel. For Vertex AI Workbench you can restart the terminal using the button on top.

In [ ]:
## Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### 🔐 Authentication
Authenticate to Google Cloud as the IAM user logged into this notebook in order to access your Google Cloud Project.

* If you are using Colab to run this notebook, use the cell below and continue.
* If you are using Vertex AI Workbench, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [ ]:
from google.colab import auth

auth.authenticate_user()

### ☁ Set Your Google Cloud Project
Set your Google Cloud project so that you can leverage Google Cloud resources within this notebook.

If you don't know your project ID, try the following:

* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113).

In [ ]:
# @markdown Please fill in the value below with your Google Cloud project ID and then run the cell.

PROJECT_ID = "my-project"  # @param {type:"string"}

# Set the project id
!gcloud config set project {PROJECT_ID}
%env GOOGLE_CLOUD_PROJECT={PROJECT_ID}

### 💡 API Enablement
The `langchain-google-spanner` package requires that you [enable the Spanner API](https://console.cloud.google.com/flows/enableapi?apiid=spanner.googleapis.com) in your Google Cloud Project.

In [ ]:
# enable Spanner API
!gcloud services enable spanner.googleapis.com

## Usage

### Set Spanner database values
Find your database values, in the [Spanner Instances page](https://console.cloud.google.com/spanner?_ga=2.223735448.2062268965.1707700487-2088871159.1707257687).

In [ ]:
# @title Set Your Values Here { display-mode: "form" }
INSTANCE = "my-instance"  # @param {type: "string"}
DATABASE = "my-database"  # @param {type: "string"}
GRAPH_NAME = "my_graph"  # @param {type: "string"}

### SpannerGraphStore

To initialize the `SpannerGraphStore` class you need to provide 3 required arguments and other arguments are optional and only need to pass if it's different from default ones

1.   a Spanner instance id;
2.   a Spanner database id belongs to the above instance id;
3.   a Spanner graph name used to create a graph in the above database.

In [ ]:
from langchain_google_spanner import SpannerGraphStore

graph_store = SpannerGraphStore(
    instance_id=INSTANCE,
    database_id=DATABASE,
    graph_name=GRAPH_NAME,
)

#### Add Graph Documents
To add graph documents in the graph store.

In [ ]:
# @title Load Documents
from langchain_core.documents import Document

text_snippets = [
    [
        """
  Product: SkyHawk Zephyr Drone
  Price: $129.99
  Weight: 220g (7.8 oz)
  The SkyHawk Zephyr is the perfect drone for beginners. It's built for effortless flying, offering a smooth and enjoyable experience from the moment you unpack it.
  Features:
  Simple Controls: Beginner friendly and intuitive controls, plus automatic takeoff and landing.
  Tough Build: Designed to handle rookie mistakes, thanks to its robust construction.
  Capture Memories: Record crisp HD photos and videos from above.
  Extended Fun: Enjoy up to 15 minutes of flight time per charge.
  Worry-Free Flying: Free Fly mode lets you fly without directional concerns.
  Take your first flight with the SkyHawk Zephyr and discover the joy of aerial views!

  Category: Drone
  Segment: ['Home']
  Tags: ['Photography', 'Videography']
""",
        """
  Bundle:SkyHawk Zephyr Starter Package
  Price: $129.99
  Everything you need to begin your drone journey:
  This package includes the essentials to get you flying and capturing stunning aerial footage. Perfect for first-time pilots who want a user-friendly and reliable drone.
  Package Includes:
  SkyHawk Zephyr Drone
  Remote Controller
  Rechargeable Battery
  USB Charging Cable
  Spare Propellers
  User Manual
  Take flight with the SkyHawk Zephyr Starter Package!
  Tags: ['Photography', 'Videography']
""",
        """
  Bunde: SkyHawk Zephyr Explorer Package
  Price: $179.99
  Enhance your flight experience with added value and accessories:
  This package is ideal for pilots who want to extend their flight time and enhance their drone's safety. It includes valuable accessories that elevate your aerial adventures.
  Package Includes:
  SkyHawk Zephyr Drone
  Remote Controller
  Rechargeable Battery + Extended Battery
  USB Charging Cable
  Spare Propellers
  Propeller Guards
  User Manual
  Carrying Case
  Explore new heights with the SkyHawk Zephyr Explorer Package!
  Tags: ['Photography', 'Videography']
""",
        """
  Product: SkyHawk Zephyr Extended Battery
  Price: $29.99
  Weight: 70g (2.5 oz)
  Fly Longer, Explore More:
  Don't let battery life limit your aerial explorations. This extended battery for the SkyHawk Zephyr drone provides significantly more flight time, allowing you to capture more breathtaking footage and explore further.
  Features:
  Increased Capacity: Enjoy up to 25 minutes of flight time on a single charge, a 67% increase over the standard battery.
  Easy Installation: Seamlessly swap out with the original battery in seconds.
  Safe and Reliable: Built to the same high-quality standards as the SkyHawk Zephyr drone.
  Extend your flight time and capture more memories with the SkyHawk Zephyr Extended Battery!
  Accessory Of:
  SkyHawk Zephyr Drone
  Tags: ['Photography', 'Videography']
""",
        """
  Product: SkyHawk Zephyr Propeller Guards
  Price: $14.99
  Weight: 20g (0.7 oz)
  Fly with Confidence:
  Protect your SkyHawk Zephyr drone and those around you with these lightweight and durable propeller guards.
  Features:
  Enhanced Safety: Prevent accidental damage to the propellers and surroundings during flight.
  Lightweight Design: Minimal impact on flight performance.
  Easy Installation: Snap on and off in seconds for quick and convenient use.
  Increased Durability: Provides an extra layer of protection for your drone.
  Fly worry-free with the SkyHawk Zephyr Propeller Guards!
  Accessory Of:
  SkyHawk Zephyr Drone
  Tags: ['Photography', 'Videography']
""",
        """
  Product: SkyHawk Zephyr 2.0 - Reach New Heights
  Price: $199.99
  Weight: 250g (8.8 oz)
  Soar to New Heights:

  This is an upgrade from SkyHawk Zephyr Drone
  The SkyHawk Zephyr 2.0 builds upon the success of its predecessor,
  offering more advanced features and improved performance for an even more incredible flying experience.
  Features:
  4K Camera: Capture breathtaking aerial footage in stunning 4K resolution with enhanced image stabilization.
  Extended Flight Time: Enjoy up to 20 minutes of flight time on a single charge.
  Increased Range: Fly farther with an extended control range of up to 1.2km (3/4 mile).
  Intelligent Flight Modes: Explore creative filming options with Follow Me, Orbit, and Waypoint modes.
  GPS-Assisted Flight: Benefit from precise positioning and enhanced safety features like Return-to-Home.
  Upgraded Motors: Experience smoother, more responsive flight and increased agility.
  Streamlined Design: A sleek and aerodynamic design for improved flight performance.
  Upgrade your aerial photography and videography with the SkyHawk Zephyr 2.0!
  Category: 'Drone'
  Segment: ['Home']
  Tags: ['Photography', 'Videography']
""",
        """
  Limited-Time Offer: SkyHawk Zephyr Drone
  Save on your first drone
  Deal Price: $109.99  (Original Price: $129.99)
  Offer valid till 31st May 2025 or while supplies last.
  Tags: ['Photography', 'Videography']
""",
        """
  Limited-Time Offer: SkyHawk Zephyr Propeller Guards
  Fly with Confidence & Save!
  Deal Price: $9.99  (Original Price: $14.99)
  You Save: 33%
  Offer valid till 31st May 2025 or while supplies last.
  Tags: ['Photography', 'Videography']
""",
    ],
    [
        """
  Product: SnapShot Voyager Camera 3.0
  Price: $249.99
  Weight: 350g (12.3 oz)
  Capture Your World with Clarity:
  The SnapShot Voyager is the ideal camera for both aspiring photographers and casual users. Designed for ease of use and exceptional image quality, it allows you to capture your memories with stunning detail.
  Features:
  Intuitive Interface: Navigate settings and modes effortlessly with a user-friendly touchscreen and simple controls.
  High-Resolution Sensor: Capture sharp, vibrant photos and videos with a 24-megapixel sensor.
  Optical Image Stabilization: Reduce blur from camera shake for clear images even in low light or while in motion.
  4K Video Recording: Record smooth, high-definition video at 4K resolution.
  Built-in Wi-Fi: Easily transfer photos and videos to your smartphone or tablet for sharing.
  Versatile Shooting Modes: Choose from a variety of modes, including portrait, landscape, sports, and night mode.
  Long Battery Life: Enjoy extended shooting sessions with a long-lasting rechargeable battery.
  Embark on your photographic journey with the SnapShot Voyager!
  Category: 'Camera'
  Segment: ['Home']
  Tags: ['Photography']
""",
        """
  Bundle: SnapShot Voyager Photography Essentials
  Price: $299.99
  Everything you need to start capturing stunning images:
  This package includes the essentials to get you started with photography. Perfect for beginners and enthusiasts who want a versatile and easy-to-use camera with the necessary accessories.
  Package Includes:
  SnapShot Voyager Camera 3.0
  16GB SD Memory Card
  Camera Carrying Case
  Rechargeable Battery
  USB Charging Cable
  Lens Cleaning Cloth
  User Manual
  Unleash your creative potential with the SnapShot Voyager Photography Essentials!
  Tags: ['Photography']
""",
        """
  Bundle: SnapShot Voyager Creative Studio Kit
  Price: $349.99
  Everything you need to expand your photographic creativity:
  This package includes accessories to help you take your photography to the next level. Perfect for enthusiasts who want to experiment with different techniques and capture more professional-looking images.
  Package Includes:
  SnapShot Voyager Camera 3.0
  32GB SD Memory Card
  Mini Tripod
  UV Protection Filter
  Polarizing Filter
  Camera Bag
  Rechargeable Battery
  USB Charging Cable
  Lens Cleaning Kit
  User Manual
  Unlock your artistic vision with the SnapShot Voyager Creative Studio Kit!
  Tags: ['Photography']
""",
        """
  Product: SnapShot Voyager Lens Hood
  Price: $19.99
  Weight: 30g (1.1 oz)
  Capture Images with Enhanced Clarity:
  Reduce glare and lens flare with the SnapShot Voyager Lens Hood. This lightweight and durable accessory helps you achieve better contrast and richer colors in your photos.
  Features:
  Improved Image Quality: Minimizes unwanted light from entering the lens, reducing glare and lens flare.
  Lens Protection: Provides added protection to the front of your lens from accidental bumps and scratches.
  Durable Construction: Made from high-quality materials for long-lasting use.
  Easy Attachment: Screws securely onto the lens for a snug and reliable fit.
  Enhance your photography with the SnapShot Voyager Lens Hood!
  Accessory Of:
  SnapShot Voyager Camera 3.0
  Tags: ['Photography']
""",
        """
  Product: SnapShot Voyager Remote Shutter Release
  Price: $24.99
  Weight: 40g (1.4 oz)
  Capture Blur-Free Images with Ease:
  Eliminate camera shake and capture sharp, clear photos, especially in low-light conditions or when using slow shutter speeds, with the SnapShot Voyager Remote Shutter Release.
  Features:
  Reduced Camera Shake: Allows you to trigger the shutter without touching the camera, minimizing movement and ensuring sharper images.
  Convenient Control: Provides a comfortable and ergonomic grip with a simple button to activate the shutter.
  Long Cable: Offers flexibility and ease of use with a generous cable length.
  Easy Connection: Plugs directly into the camera's remote port for quick and simple setup.
  Capture perfect moments with the SnapShot Voyager Remote Shutter Release!
  Accessory Of:
  SnapShot Voyager Camera 3.0
  Tags: ['Photography']
""",
        """
  Limited-Time Offer: SnapShot Voyager Remote Shutter Release
  Capture Blur-Free Images & Save!
  Deal Price: $16.99 (Original Price: $24.99)
  You Save: 32%
  Offer valid till 28th February 2025 or while supplies last.
  Tags: ['Photography']
""",
    ],
    [
        """
## AuraBook X500 - Quantum Leaps in Performance

**Product Summary:**

Experience the future of computing with the AuraBook X500, a revolutionary laptop powered by cutting-edge quantum processing. Designed for professionals and enthusiasts alike, the AuraBook X500 delivers unparalleled speed, security, and efficiency, effortlessly handling demanding tasks and complex applications.

**Price:** $1499.00

**Manufacturer:** AuraTech Innovations

**Key Features:**

*   **Quantum Processor:** Experience exponential performance gains over traditional processors.
*   **16GB Quantum RAM:** Ensures smooth multitasking and responsiveness.
*   **1TB NVMe SSD:** Lightning-fast storage for your files and applications.
*   **15.6" 4K UHD Display:** Immersive visuals with vibrant colors and sharp detail.
*   **Advanced Security:** Quantum-enhanced encryption safeguards your data.
*   **Long-lasting Battery:** Stay productive on the go with up to 12 hours of battery life.
*   **Sleek and Lightweight Design:** Crafted from premium aluminum for a stylish and portable design.
*   **Wi-Fi 6E & Bluetooth 5.3:** Stay connected with the latest wireless technology.

**Specifications:**

*   **Processor:** AuraTech Quantum Core Q1
*   **Memory:** 16GB Quantum RAM (Expandable to 32GB)
*   **Storage:** 1TB NVMe PCIe Gen4 SSD
*   **Display:** 15.6" 4K UHD (3840 x 2160) IPS Anti-Glare Display, 500 nits
*   **Graphics:** Integrated Quantum Graphics Processor
*   **Operating System:** AuraOS (based on Linux)
*   **Ports:** 2 x USB-C (Thunderbolt 4), 1 x USB-A 3.2, 1 x HDMI 2.1, 1 x 3.5mm Audio Jack
*   **Wireless:** Wi-Fi 6E (802.11ax), Bluetooth 5.3
*   **Webcam:** 1080p FHD Webcam with Privacy Shutter
*   **Audio:** Integrated Stereo Speakers with Quantum Audio Processing
*   **Battery:** 80Wh Lithium-Polymer Battery
*   **Dimensions:** 14.0" x 9.5" x 0.7"
*   **Weight:** 3.5 lbs
*   **Color:** Quantum Silver

**Compatible Accessories (Sold Separately):**

*   PrivacyGuard Pro
*   Lumix Keyboard
Category: Laptops
Segment: ['Home', 'Office']
Tags: ['Computer']
""",
        """
Product Summary: SecureView Privacy Screen

Price: $79.00

Manufacturer: AuraTech (Designed for AuraBook)

Description: Protect your sensitive information and work discreetly with the SecureView Privacy Screen. This easy-to-install screen attaches seamlessly to your laptop, limiting the viewing angle to +/- 30 degrees. Only you can see your screen clearly, while those around you see a darkened view. Ideal for travel, open office environments, and protecting confidential data.

Features:

* Privacy Protection: Keeps your screen content private from onlookers.
* Anti-Glare: Reduces glare and reflections for comfortable viewing.
* Easy Installation: Attaches quickly and easily with reusable adhesive strips.
* Scratch Resistant: Durable coating protects your screen from scratches and smudges.
* Lightweight & Thin: Maintains the sleek profile of your laptop.
* Protects your screen from dust and scratches.

Specifications:

* Compatibility: AuraBook X500, AuraBook Air
* Dimensions: Precisely sized to fit the AuraBook X500 and AuraBook Air screens.
* Material: Premium multi-layered PET film
* Viewing Angle: 60 degrees total (30 degrees from center on each side)
* Thickness: 0.4mm
* Attachment Method: Reusable adhesive strips (included)
* Color: Black
Category: Laptop Privacy Screen
Segment: ['Home', 'Office']
Tags: ['Computer']

""",
        """
**Product Summary: AuraBook Air**

Stay productive on the move with the AuraBook Air, a lightweight and powerful laptop designed for ultimate portability.

Price: $999.00

Manufacturer: Stellaris Systems

Description: The AuraBook Air is engineered for users who demand performance without the bulk. Its sleek design and robust internal components make it the perfect companion for work, travel, and entertainment.

Features:

*   Ultra-lightweight design for maximum portability
*   Brilliant 13.3-inch Full HD Display
*   Fast and responsive performance
*   Long-lasting battery life for all-day use
*   Integrated HD webcam for video conferencing
*   Sleek, modern design

Specifications:

*   Processor: Intel Core i5-1135G7 Processor
*   Memory: 8GB LPDDR4X RAM
*   Storage: 256GB NVMe PCIe SSD
*   Graphics: Intel Iris Xe Graphics
*   Display: 13.3-inch Full HD (1920 x 1080) IPS Display
*   Operating System: Windows 11 Home
*   Ports: 2 x Thunderbolt 4, 1 x USB 3.2 Gen 1, 1 x 3.5mm Headphone Jack
*   Wireless: Wi-Fi 6 (802.11ax), Bluetooth 5.1
*   Battery: Up to 12 hours
*   Weight: 2.5 lbs
*   Dimensions: 11.97" x 8.36" x 0.63"
*   Color: Space Gray

Accessories: Enhance your AuraBook Air experience with compatible accessories like the PrivacyGuard Standard screen protector and the Lumix Keyboard (sold separately).
Category: Laptops
Segment: ['Home', 'Office']
Tags: ['Computer']
""",
        """
Product Summary: PrivacyGuard Pro

Shield your screen from prying eyes and protect yourself from harmful blue light with the PrivacyGuard Pro. This advanced privacy screen offers superior clarity while ensuring your on-screen data remains confidential. Upgrade from the SecureView Privacy Screen, the PrivacyGuard Pro delivers enhanced protection and viewing comfort. Perfect accessory for your AuraBook X500.

Price: $99.00

Manufacturer: Visionary Displays Inc.

Features:

*   Advanced Privacy Technology: Restricts viewing angle to +/- 30 degrees.
*   Blue Light Filter: Reduces harmful blue light emissions by up to 65%.
*   Anti-Glare Coating: Minimizes reflections for comfortable viewing in bright environments.
*   Scratch Resistant: Protects your screen from scratches and scuffs.
*   Easy Application: Bubble-free adhesive for simple and secure attachment.
*   Case-Friendly Design: Compatible with most AuraBook X500 cases.

Specifications:

*   Material: Premium PET Film with Multi-Layer Coating
*   Thickness: 0.4mm
*   Compatibility: AuraBook X500
*   Weight: 25g
*   Dimensions: 304.1 mm x 217.6 mm
*   Color: Clear
*   Package Contents: PrivacyGuard Pro screen protector, Microfiber cleaning cloth, Installation guide, Dust removal stickers.
Category: Laptop Privacy Screen
Segment: ['Home', 'Office']
Tags: ['Computer']
""",
        """
Lumix Keyboard

Price: $129.00

Manufacturer: Lumix Tech

Description: Experience unparalleled comfort and style with the Lumix Keyboard. Designed for ergonomic excellence, this keyboard minimizes strain and maximizes productivity. Immerse yourself in a world of color with customizable backlighting, allowing you to personalize your workspace to match your mood or gaming setup.

Features:

*   Ergonomic Design: Contoured shape promotes natural wrist and hand positioning, reducing fatigue during long typing sessions.
*   Customizable RGB Backlighting: Choose from millions of colors and dynamic lighting effects to create a personalized ambiance.
*   Durable Construction: Built to last with high-quality materials and a robust frame.
*   Responsive Keys: Enjoy a tactile and responsive typing experience with precise key actuation.
*   Plug and Play: Simple USB connection for instant compatibility with Windows and macOS.
*   Anti-Ghosting Technology: Ensures every keystroke is registered accurately, even during fast-paced gaming.
*   Dedicated Media Controls: Easily control your music and videos with convenient media buttons.

Specifications:

*   Model Number: LK-720
*   Connectivity: USB 2.0
*   Keycap Material: ABS Plastic
*   Dimensions: 17.5 x 5.5 x 1.5 inches
*   Weight: 1.8 pounds
*   Cable Length: 6 feet
*   Compatibility: Windows 10/8/7, macOS 10.12 or later
*   Backlight: Customizable RGB
*   Number of Keys: 104
*   Accessory Of: AuraBook X500, AuraBook Air
Category: Accessories
Segment: ['Home', 'Office']
Tags: ['Computer']
""",
        """
PrivacyGuard Standard

Price: $49.00

Manufacturer: Privacy Solutions Inc.

Description: Keep your on-screen information private with the PrivacyGuard Standard screen protector. Designed for optimal viewing clarity directly in front of the screen while blocking visibility from side angles, this screen protector is perfect for working in public spaces, commuting, or protecting sensitive information.

Features:

*   Blocks side views for privacy
*   Anti-glare finish reduces eye strain
*   Scratch-resistant coating protects your screen
*   Easy to apply and remove
*   Bubble-free installation

Specifications:

*   Compatibility: AuraBook Air
*   Material: PET Film with Privacy Layer
*   Thickness: 0.4mm
*   Transparency: 65%
*   Viewing Angle: 60 degrees (30 degrees left/right)
*   Package Contents: PrivacyGuard Standard, cleaning cloth, application instructions.
Category: Accessories
Segment: ['Home', 'Office']
Tags: ['Computer']
""",
        """
Product Summary: Quantum Charger

Stay powered up and ready to go with the Quantum Charger, designed for unparalleled charging speed and reliability. This super fast charger significantly reduces downtime, letting you get back to what matters most. Specifically optimized for and fully compatible with the AuraBook X500.

Price: $69.00

Manufacturer: Quantum Dynamics

Features:

*   Super Fast Charging Technology: Provides significantly faster charging speeds compared to standard chargers.
*   Optimized for AuraBook X500: Guarantees seamless compatibility and optimal performance with the AuraBook X500.
*   Compact and Portable Design: Easily fits in your bag or backpack for charging on the go.
*   Built-in Safety Features: Over-current, over-voltage, and short-circuit protection ensures safe and reliable charging.
*   LED Indicator Light: Displays charging status at a glance.

Specifications:

*   Input: 100-240V AC, 50/60Hz
*   Output: 20V/3.25A (65W Max)
*   Connector Type: USB-C
*   Cable Length: 6 ft
*   Weight: 0.25 lbs
*   Color: Space Gray
*   Certifications: UL, CE, FCC, RoHS
Category: Accessories
Segment: ['Home', 'Office']
Tags: ['Computer']
""",
        """
Office Bundle

Price: $1600

Manufacturer: Aura Systems

Description: Enhance your productivity with this comprehensive office bundle, featuring a powerful laptop and essential accessories designed for seamless workflow.

Features:

AuraBook Air:
    Ultra-thin and lightweight design for portability.
    14-inch Full HD IPS Display with anti-glare coating.
    Intel Core i5-1235U Processor.
    8GB LPDDR5 RAM.
    512GB NVMe SSD storage.
    Integrated Intel Iris Xe Graphics.
    Up to 12 hours of battery life.
    Windows 11 Home.

Lumix Keyboard:
    Ergonomic design for comfortable typing.
    Wireless Bluetooth 5.0 connection.
    Full-size layout with number pad.
    Quiet Scissor-Switch Keys.
    Integrated rechargeable battery with up to 3 months of battery life.
    Compatible with Windows, macOS, iOS, and Android.

PrivacyGuard Standard:
    14-inch Anti-glare privacy screen filter.
    Limits viewing angle to 60 degrees.
    Protects sensitive information from prying eyes.
    Easy to attach and remove.
    Reduces blue light exposure.

Specifications:

AuraBook Air:
    Dimensions: 12.3 x 8.5 x 0.6 inches
    Weight: 2.5 lbs
    Ports: 2 x USB-C, 1 x USB-A, 1 x HDMI, 1 x 3.5mm headphone jack

Lumix Keyboard:
    Dimensions: 17.2 x 4.9 x 0.8 inches
    Weight: 1.2 lbs

PrivacyGuard Standard:
    Size: 14 inch (diagonal)
    Aspect Ratio: 16:9
Category: Bundles
Segment: ['Home', 'Office']
Tags: ['Computer']
""",
        """
Privacy Protection Bundle

Price: $1550.00

Manufacturer: Aura Technologies (AuraBook Air) & SecureView (SecureView Privacy Screen)

Description: The ultimate privacy and security package, bundling the powerful AuraBook Air laptop with the SecureView Privacy Screen for on-the-go protection against visual hacking. Work confidently knowing your data is safe from prying eyes, whether you're in a crowded cafe or a busy airport.

Features:

*   Complete Privacy Solution: Combines hardware and visual security for maximum protection.
*   Enhanced Productivity: AuraBook Air delivers exceptional performance.
*   On-the-Go Security: SecureView Privacy Screen ensures confidentiality in public spaces.
*   Easy Installation: The SecureView Privacy Screen easily attaches and detaches.
*   Sleek Design: Maintain a professional look while prioritizing security.

AuraBook Air Specifications:

*   Processor: Intel Core i7-1260P
*   Memory: 16GB LPDDR5 RAM
*   Storage: 512GB NVMe PCIe Gen4 SSD
*   Display: 13.3-inch IPS Anti-Glare Display, 2560 x 1600 Resolution
*   Operating System: Windows 11 Pro
*   Graphics: Intel Iris Xe Graphics
*   Ports: 2 x Thunderbolt 4, 1 x USB 3.2 Gen 1, 1 x HDMI 2.0, 1 x 3.5mm Headphone Jack
*   Wireless: Wi-Fi 6E, Bluetooth 5.2
*   Battery Life: Up to 12 hours (depending on usage)
*   Weight: 2.8 lbs
*   Color: Space Gray

SecureView Privacy Screen Specifications:

*   Compatibility: Specifically designed for the AuraBook Air 13.3-inch model
*   Viewing Angle: Restricts viewing to +/- 30 degrees
*   Attachment: Reusable adhesive strips for easy on/off application
*   Material: Multi-layered optical film
*   Scratch Resistance: Hard-coated surface to resist scratches and fingerprints
*   Dimensions: Precisely cut to fit the AuraBook Air screen
Category: Bundles
Segment: ['Home', 'Office']
Tags: ['Computer']

""",
        """
**AuraBook X500 Deal - Limited Time Offer!**

**Price:** $1400.00

**Manufacturer:** Aura Technologies

**Description:**

Secure the powerhouse AuraBook X500 at a special discounted price for a limited time only! This high-performance laptop is perfect for demanding tasks, creative projects, and immersive entertainment. Don't miss out on this incredible offer to own the AuraBook X500 at a significantly reduced cost.

**Features:**

*   Stunning 15.6-inch Full HD IPS Display
*   Intel Core i7-1260P Processor
*   16GB DDR5 RAM
*   512GB NVMe Solid State Drive
*   NVIDIA GeForce RTX 3050 Graphics Card
*   Backlit Keyboard
*   Integrated HD Webcam with Privacy Shutter
*   Wi-Fi 6 and Bluetooth 5.2 Connectivity
*   Thunderbolt 4 Ports for Fast Data Transfer
*   Lightweight and Durable Aluminum Chassis
*   Windows 11 Home

**Specifications:**

*   Operating System: Windows 11 Home
*   Display: 15.6" Full HD (1920 x 1080) IPS Display
*   Processor: Intel Core i7-1260P (Up to 4.7 GHz, 18 MB Intel Smart Cache)
*   Memory: 16GB DDR5 4800MHz RAM
*   Storage: 512GB NVMe PCIe Gen4 SSD
*   Graphics: NVIDIA GeForce RTX 3050 4GB GDDR6
*   Wireless: Wi-Fi 6 (802.11ax)
*   Bluetooth: Bluetooth 5.2
*   Ports: 2 x Thunderbolt 4, 2 x USB 3.2 Gen 1, 1 x HDMI, 1 x Headphone/Microphone Combo Jack, 1 x SD Card Reader
*   Webcam: 720p HD Webcam with Privacy Shutter
*   Audio: Stereo Speakers
*   Battery: 60Wh Lithium-Ion Battery
*   Dimensions: 14.1 x 9.3 x 0.7 inches
*   Weight: 3.9 pounds
*   Color: Space Gray

**Limited Time Offer:** This discounted price is only valid until March 15, 2024. Order yours today!
Category: Deals
Segment: ['Home', 'Office']
Tags: ['Computer']
""",
        """
Product Summary: AuraView 32 Curved Monitor

Immerse yourself in stunning visuals with the AuraView 32, a 32-inch curved monitor designed for exceptional clarity and vibrant color reproduction. Whether you're gaming, working, or streaming, the AuraView 32 delivers an immersive and engaging experience.

Price: $399.99

Manufacturer: Aura Displays

Features:

*   32-inch Curved Display: Experience enhanced immersion with a gentle curve that wraps around your field of vision.
*   Vibrant Color Display: Enjoy rich, accurate colors thanks to a wide color gamut and excellent color calibration.
*   Flicker-Free Technology: Reduces eye strain during extended use.
*   Blue Light Filter: Minimizes harmful blue light emissions for comfortable viewing.
*   Multiple Ports: Includes HDMI and DisplayPort inputs for versatile connectivity.
*   Built-in Speakers: Integrated speakers provide convenient audio without the need for external devices.
*   VESA Mount Compatible: Easily mount the monitor on a wall or monitor arm for optimal positioning.
*   Sleek, Modern Design: Complements any desktop setup.

Specifications:

*   Screen Size: 32 inches
*   Panel Type: VA
*   Resolution: 1920 x 1080 (Full HD)
*   Aspect Ratio: 16:9
*   Response Time: 5ms (GTG)
*   Refresh Rate: 75Hz
*   Brightness: 250 cd/m²
*   Contrast Ratio: 3000:1 (Typical)
*   Viewing Angles: 178° (Horizontal/Vertical)
*   Connectivity: 1 x HDMI, 1 x DisplayPort, 1 x 3.5mm Audio Out
*   Power Consumption: 45W (Typical)
*   Dimensions (with stand): 28.1" x 20.5" x 8.7"
*   Weight (with stand): 12.3 lbs

Related Accessory: The AuraView 32 is fully compatible with the ErgoFlex Pro monitor arm for enhanced ergonomic adjustability.
Category: Monitors
Segment: ['Home', 'Office']
Tags: ['Computer']
""",
        """
**Product Summary: AuraView 32 Pro**

Immerse yourself in stunning visuals with the AuraView 32 Pro monitor. Building upon the success of the AuraView 32, this enhanced version delivers exceptional brightness and a silky-smooth refresh rate, perfect for gaming, content creation, and everyday use.

**Price:** $599.99

**Manufacturer:** Aura Displays

**Features:**

*   Enhanced Brightness: Experience vibrant colors and deeper blacks with increased brightness levels.
*   High Refresh Rate: Enjoy fluid motion and reduced blur with a 165Hz refresh rate.
*   Adaptive Sync Technology: Eliminate screen tearing and stuttering for a seamless gaming experience.
*   Ultra-Thin Bezel Design: Maximize your viewing area and create a multi-monitor setup with minimal distractions.
*   Built-in Speakers: Integrated 2 x 3W speakers provide convenient audio output.
*   Multiple Connectivity Options: Includes HDMI 2.0, DisplayPort 1.4, and USB ports.

**Specifications:**

*   Screen Size: 32 inches
*   Panel Type: IPS
*   Resolution: 2560 x 1440 (QHD)
*   Aspect Ratio: 16:9
*   Response Time: 1ms (MPRT)
*   Brightness: 400 cd/m²
*   Contrast Ratio: 1000:1 (Typical)
*   Viewing Angle: 178° (H) / 178° (V)
*   Color Gamut: 99% sRGB
*   VESA Mount: 100 x 100 mm
*   Weight: 12 lbs (with stand)

**Related Accessories:**

Enhance your viewing experience with the ErgoFlex Pro monitor arm (sold separately).
Category: Monitors
Segment: ['Home', 'Office']
Tags: ['Computer']
""",
        """
ErgoFlex Pro - Premium Monitor Arm

Price: $149.99

Manufacturer: FlexiTech Solutions

Description: Experience unparalleled ergonomic comfort and productivity with the ErgoFlex Pro monitor arm. Designed for optimal viewing and posture, the ErgoFlex Pro allows you to effortlessly adjust your monitor's height, tilt, and swivel, reducing strain on your neck, back, and eyes.

Features:

Full Articulation: Enjoy a wide range of motion with adjustable height, tilt ( -85° to +90°), swivel (±90°), and rotation (360°).

Gas Spring System: Provides smooth and effortless monitor adjustments.

Cable Management: Integrated cable management system keeps your workspace clean and organized.

Sturdy Construction: Built with high-quality aluminum and steel for maximum stability and durability. Supports monitors up to 32 inches and 17.6 lbs.

Easy Installation: Includes both desk clamp and grommet mount options for easy setup.

Specifications:

Compatibility: Supports monitors from 17" to 32"

Weight Capacity: 4.4 lbs to 17.6 lbs (2 kg to 8 kg)

VESA Compatibility: 75x75mm and 100x100mm

Material: Aluminum, Steel, Plastic

Tilt: -85° to +90°

Swivel: ±90°

Rotation: 360°

Height Adjustment Range: 7.9" to 19.7" (200mm to 500mm)

Mounting Options: Desk Clamp or Grommet Mount

Color: Matte Black

Included in the Box: Monitor Arm, Mounting Hardware, Installation Guide

Accessory Of: AuraView 32, AuraView 32 Pro, Cable Management Kit
Category: Monitor Arms
Segment: ['Home', 'Office']
Tags: ['Computer']
""",
        """
Cable Management Kit

Price: $29.99

Manufacturer: ErgoDynamics

Description: Eliminate cable clutter and maintain a clean, professional workspace with our Cable Management Kit. Designed to seamlessly integrate with your ErgoFlex Pro monitor arm (but compatible with most desks and monitor setups), this kit provides everything you need to keep your monitor cables tidy and organized.

Features:

*   Comprehensive Kit: Includes cable ties, cable clips, and a flexible cable sleeve.
*   Easy Installation: Simple, tool-free installation using adhesive backing and adjustable straps.
*   Durable Materials: Made from high-quality, long-lasting materials for reliable performance.
*   Sleek Design: Aesthetically pleasing design that complements any office environment.
*   Reusable and Adjustable: Reconfigure your cable management as needed.

Specifications:

*   Material: Durable ABS plastic (clips), Nylon (sleeve), Velcro (ties)
*   Color: Black
*   Cable Sleeve Length: 24 inches
*   Cable Clip Quantity: 6
*   Cable Tie Quantity: 10
*   Adhesive: 3M VHB tape
*   Weight: 0.25 lbs
*   Accessory Of: ErgoFlex Pro Monitor Arm
Category: Accessories
Segment: ['Home', 'Office']
Tags: ['Computer']

""",
        """
Product Summary:

Product Name: Blue Light Filter Screen

Price: $39.99

Description: Combat eye strain and fatigue with our premium Blue Light Filter Screen. Designed to significantly reduce harmful blue light emitted from your monitor, this screen protector helps minimize eye strain, headaches, and disrupted sleep patterns caused by prolonged screen use. Enjoy greater comfort and productivity, whether you're working, gaming, or simply browsing.

Manufacturer: VisionGuard Tech

Features:

*   Reduces Blue Light Emission: Filters out a significant percentage of harmful blue light.
*   Anti-Glare Coating: Minimizes reflections and glare for improved visibility.
*   Scratch Resistant: Protects your monitor screen from scratches and scuffs.
*   Easy Installation: Simple clip-on design for quick and hassle-free attachment.
*   Crystal Clear Clarity: Maintains optimal screen clarity and image quality.
*   Lightweight and Durable: Made from high-quality acrylic material.

Specifications:

*   Screen Size Compatibility: Specifically designed for 32-inch monitors.
*   Dimensions: 27.9 inches (width) x 15.7 inches (height)
*   Material: Acrylic
*   Blue Light Reduction: Up to 60%
*   Weight: 1.2 lbs
*   Compatibility: Perfectly fits AuraView 32 and AuraView 32 Pro monitors. Also compatible with most standard 32" monitors.
Category: Accessories
Segment: ['Home', 'Office']
Tags: ['Computer']

""",
        """
VESA Mount Adapter

Price: $19.99

Description: Easily adapt monitors without standard VESA mounting holes to VESA-compatible monitor arms and stands. This adapter allows you to use your existing non-VESA monitor with a wide range of monitor mounts, saving you the cost of a new display. Specifically designed as an accessory for the ErgoFlex Pro monitor arm, but compatible with many other mounts (check specifications below for compatibility).

Manufacturer: Mount Solutions Inc.

Features:

*   Universal Compatibility: Supports most non-VESA monitors from 17" to 27" in size.
*   Sturdy Steel Construction: Provides a secure and stable connection.
*   Easy Installation: Includes all necessary hardware and instructions.
*   Scratch-Resistant Finish: Protects your monitor and adapter.
*   Low Profile Design: Minimizes the distance between the monitor and mount.

Specifications:

*   Material: Powder-Coated Steel
*   VESA Compatibility: 75x75mm, 100x100mm
*   Monitor Size Compatibility: 17" - 27" (Diagonal Measurement)
*   Monitor Thickness Compatibility: 1" - 2.75"
*   Weight Capacity: 17.6 lbs (8 kg)
*   Color: Black
*   Product Dimensions: 17.7" x 4.7" x 0.2"
*   UPC: 728028891217
Category: Accessories
Segment: ['Home', 'Office']
Tags: ['Computer']

""",
        """
Dual Monitor Productivity Bundle

Price: $849.99

Manufacturer: Aura Displays

Description: Supercharge your productivity with the Dual Monitor Productivity Bundle, featuring two stunning AuraView 32 monitors. Experience enhanced multitasking, seamless workflow, and immersive visuals for work, creative projects, or entertainment.

Features:

*   Dual Monitor Setup: Double your screen real estate for increased productivity.
*   32-inch AuraView 32 Monitors: Experience vibrant colors and sharp details.
*   Ultra-thin Bezel: Enjoy a near-seamless multi-monitor setup.
*   Wide Viewing Angle: Consistent image quality from any perspective.
*   Built-in Speakers: Integrated audio for convenient sound.
*   VESA Mount Compatible: Flexibility in mounting options.
*   Flicker-Free Technology: Reduces eye strain during prolonged use.
*   Low Blue Light Mode: Minimizes blue light emission for comfortable viewing.

Specifications (per monitor):

*   Screen Size: 32 inches
*   Panel Type: IPS
*   Resolution: 2560 x 1440 (QHD)
*   Aspect Ratio: 16:9
*   Refresh Rate: 75Hz
*   Response Time: 5ms (GTG)
*   Brightness: 300 cd/m²
*   Contrast Ratio: 1000:1 (Typical)
*   Viewing Angle (H/V): 178°/178°
*   Connectivity: 1 x HDMI 1.4, 1 x DisplayPort 1.2, 1 x 3.5mm Audio Out
*   Power Consumption: 45W (Typical)
*   Weight: 12 lbs (per monitor)
*   Dimensions (with stand): 28.5" x 20" x 8" (per monitor)

What's Included:

*   2 x AuraView 32 Monitors
*   2 x Monitor Stands
*   2 x Power Cables
*   1 x HDMI Cable
*   1 x DisplayPort Cable
*   User Manual
Category: Bundles
Segment: ['Home', 'Office']
Tags: ['Computer']

""",
        """
**Ergonomic Workstation Bundle: AuraView 32 + ErgoFlex Pro**

**Price:** $649.99

**Manufacturer:** AuraCraft

**Description:** Transform your workspace into a haven of comfort and productivity with the Ergonomic Workstation Bundle. This bundle combines the stunning visuals of the AuraView 32" monitor with the unparalleled ergonomic support of the ErgoFlex Pro adjustable standing desk. Say goodbye to aches, pains, and eye strain, and hello to a healthier, more efficient you.

**Features:**

*   **AuraView 32 Monitor:**
    *   32-inch curved display for immersive viewing
    *   Ultra HD (3840 x 2160) resolution for crystal-clear visuals
    *   Low blue light filter for reduced eye strain
    *   Built-in speakers for convenient audio
    *   Adjustable tilt and swivel for optimal positioning
*   **ErgoFlex Pro Standing Desk:**
    *   Electric height adjustment from 27 inches to 46 inches
    *   Spacious 48" x 24" work surface
    *   Sturdy steel frame supports up to 220 lbs
    *   Memory settings for personalized height preferences
    *   Cable management system to keep your workspace tidy

**Specifications:**

*   **AuraView 32 Monitor:**
    *   Panel Type: VA
    *   Response Time: 5ms
    *   Refresh Rate: 60Hz
    *   Connectivity: HDMI 2.0 x 2, DisplayPort 1.2 x 1, Audio Out
    *   VESA Mount Compatibility: 100x100mm
*   **ErgoFlex Pro Standing Desk:**
    *   Frame Material: Powder-coated steel
    *   Desktop Material: Engineered wood
    *   Motor Type: Single motor
    *   Noise Level: <50dB
    *   Weight Capacity: 220 lbs
Category: Bundles
Segment: ['Home', 'Office']
Tags: ['Computer']

""",
        """
**Product Summary: Summer Savings Deal - AuraView 32" LED TV**

**Headline:** Save Big This Summer on the AuraView 32" LED TV!

**Price:** $349.99

**Manufacturer:** AuraView

**Description:** Don't miss out on our limited-time Summer Savings Deal! Get the stunning AuraView 32" LED TV at a discounted price. Experience vibrant colors, sharp details, and immersive sound, all in a compact and stylish design. This deal ends July 31st, 2024, so act fast!

**Features:**

*   32-inch LED Display: Enjoy crisp, clear picture quality.
*   720p HD Resolution: Delivers detailed visuals for your favorite content.
*   Built-in Speakers: Integrated speakers provide rich and clear audio.
*   Multiple Ports: Includes HDMI, USB, and composite inputs for versatile connectivity.
*   Slim Bezel Design: Maximizes screen space for an immersive viewing experience.
*   Energy Efficient: Designed to consume less power, saving you money.
*   Parental Control: Keep content safe for kids.
*   Sleep Timer: Set the TV to turn off automatically.

**Specifications:**

*   Screen Size: 32 inches
*   Resolution: 1366 x 768 (720p HD)
*   Display Technology: LED
*   Aspect Ratio: 16:9
*   Refresh Rate: 60Hz
*   HDMI Ports: 2
*   USB Ports: 1
*   Composite Input: 1
*   Audio Output: 2 x 8W
*   VESA Mount Compatible: 100mm x 100mm
*   Power Consumption: 45W (Typical)
*   Dimensions (with stand): 28.7" W x 18.5" H x 7.1" D
*   Weight (with stand): 8.2 lbs
*   Model Number: AV32-LE2024

**Limited Time Offer:** This special summer pricing is only valid until July 31, 2024. Order yours today!
Category: Deals
Segment: ['Home', 'Office']
Tags: ['Computer']
""",
    ],
    [
        """
ChromaJet Pro

Price: $499.00

Manufacturer: Pixel Perfect Technologies

Description: Experience vibrant, high-resolution color printing at lightning-fast speeds with the ChromaJet Pro. Perfect for home offices, small businesses, and creative professionals, this printer delivers exceptional image quality and reliable performance.

Features:

*   High-Resolution Printing: Up to 4800 x 1200 dpi for stunning detail and clarity.
*   Fast Print Speeds: Prints up to 24 pages per minute (ppm) in black and white, and 18 ppm in color.
*   Wireless Connectivity: Seamlessly connect via Wi-Fi for convenient printing from your computer, smartphone, or tablet.
*   Automatic Duplex Printing: Save paper and resources with automatic two-sided printing.
*   Borderless Printing: Create professional-looking photos and documents without borders.
*   User-Friendly Interface: Intuitive LCD screen for easy navigation and operation.
*   Large Paper Capacity: 250-sheet input tray minimizes refills.
*   Mobile Printing: Compatible with Apple AirPrint, Google Cloud Print, and Mopria Print Service.

Specifications:

*   Print Technology: Inkjet
*   Print Resolution: 4800 x 1200 dpi (color), 1200 x 1200 dpi (black)
*   Print Speed: 22 ppm (black), 18 ppm (color)
*   Paper Size: Letter, Legal, 4x6, 5x7, 8x10, Envelopes
*   Connectivity: Wi-Fi, USB 2.0
*   Memory: 256 MB
*   Dimensions: 17.5" x 14.5" x 6.5"
*   Weight: 12 lbs
*   Operating Systems: Windows, macOS
*   Power Consumption: 20W (Operating), 1.5W (Standby)

Accessories: Enhance your ChromaJet Pro experience with these recommended accessories:

*   ChromaShield Laminator: Protect your prints with a durable, glossy finish.
*   ChromaClean Kit: Keep your printer running smoothly with this comprehensive cleaning kit.
Category: Printer
Segment: ['Home', 'Office']
Tags: ['Printing']

""",
        """
 ChromaJet Pro

Price: $499.00

Manufacturer: Pixel Perfect Technologies

Description: Experience vibrant, high-resolution color printing at lightning-fast speeds with the ChromaJet Pro. Perfect for home offices, small businesses, and creative professionals, this printer delivers exceptional image quality and reliable performance.

Features:

*   High-Resolution Printing: Up to 4800 x 1200 dpi for stunning detail and clarity.
*   Fast Print Speeds: Prints up to 22 pages per minute (ppm) in black and white, and 18 ppm in color.
*   Wireless Connectivity: Seamlessly connect via Wi-Fi for convenient printing from your computer, smartphone, or tablet.
*   Automatic Duplex Printing: Save paper and resources with automatic two-sided printing.
*   Borderless Printing: Create professional-looking photos and documents without borders.
*   User-Friendly Interface: Intuitive LCD screen for easy navigation and operation.
*   Large Paper Capacity: 250-sheet input tray minimizes refills.
*   Mobile Printing: Compatible with Apple AirPrint, Google Cloud Print, and Mopria Print Service.

Specifications:

*   Print Technology: Inkjet
*   Print Resolution: 4800 x 1200 dpi (color), 1200 x 1200 dpi (black)
*   Print Speed: 22 ppm (black), 18 ppm (color)
*   Paper Size: Letter, Legal, 4x6, 5x7, 8x10, Envelopes
*   Connectivity: Wi-Fi, USB 2.0
*   Memory: 256 MB
*   Dimensions: 17.5" x 14.5" x 6.5"
*   Weight: 12 lbs
*   Operating Systems: Windows, macOS
*   Power Consumption: 20W (Operating), 1.5W (Standby)

Accessories: Enhance your ChromaJet Pro experience with these recommended accessories:

*   ChromaShield Laminator: Protect your prints with a durable, glossy finish.
*   ChromaClean Kit: Keep your printer running smoothly with this comprehensive cleaning kit.
Category: Printer
Segment: ['Home', 'Office']
Tags: ['Printing']

""",
        """
ChromaJet Pro XL

Price: $799.00

Manufacturer: ChromaTech

Description: Unlock professional-grade printing with the ChromaJet Pro XL. Building upon the acclaimed ChromaJet Pro, the XL version expands your creative possibilities with support for larger paper sizes and enhanced color management. Achieve vibrant, accurate colors and crisp details on every print. Perfect for graphic designers, photographers, and creative professionals who demand the best.

Features:

* Supports paper sizes up to 13" x 19" (Super B/A3+)
* Advanced color management system with ICC profile support
* High-resolution printing up to 4800 x 2400 dpi
* Fast print speeds: Up to 15 ppm in black and 10 ppm in color
* Wireless connectivity (Wi-Fi & AirPrint)
* Ethernet connectivity
* 250-sheet paper tray capacity
* Individual ink cartridges for cost-effective printing
* Intuitive LCD control panel

Specifications:

* Print Technology: Inkjet
* Maximum Print Resolution: 4800 x 2400 dpi
* Print Speed (Black): 15 ppm
* Print Speed (Color): 10 ppm
* Paper Sizes Supported: Letter, Legal, 4x6, 5x7, 8x10, 11x14, 13x19 (Super B/A3+)
* Connectivity: USB 2.0, Wi-Fi, Ethernet
* Ink Cartridge Type: Individual
* Operating System Compatibility: Windows, macOS
* Dimensions: 22" x 16" x 9"
* Weight: 18 lbs

Upgrade From: ChromaJet Pro

Compatible Accessories (Sold Separately):

* ChromaShield Laminator: Protect your prints with a durable laminate finish.
* ChromaClean Kit: Keep your printer running smoothly with this comprehensive cleaning kit.
* ChromaPaper Pro Pack: Experience superior print quality with our premium photo paper.
Category: Printer
Segment: ['Home', 'Office']
Tags: ['Printing']
""",
        """
 ChromaJet Pro XL

Price: $799.00

Manufacturer: ChromaTech

Description: Unlock professional-grade printing with the ChromaJet Pro XL. Building upon the acclaimed ChromaJet Pro, the XL version expands your creative possibilities with support for larger paper sizes and enhanced color management. Achieve vibrant, accurate colors and crisp details on every print. Perfect for graphic designers, photographers, and creative professionals who demand the best.

Features:

* Supports paper sizes up to 13" x 19" (Super B/A3+)
* Advanced color management system with ICC profile support
* High-resolution printing up to 4800 x 2400 dpi
* Fast print speeds: Up to 15 ppm in black and 10 ppm in color
* Wireless connectivity (Wi-Fi & AirPrint)
* Ethernet connectivity
* 250-sheet paper tray capacity
* Individual ink cartridges for cost-effective printing
* Intuitive LCD control panel

Specifications:

* Print Technology: Inkjet
* Maximum Print Resolution: 4800 x 2400 dpi
* Print Speed (Black): 15 ppm
* Print Speed (Color): 10 ppm
* Paper Sizes Supported: Letter, Legal, 4x6, 5x7, 8x10, 11x14, 13x19 (Super B/A3+)
* Connectivity: USB 2.0, Wi-Fi, Ethernet
* Ink Cartridge Type: Individual
* Operating System Compatibility: Windows, macOS
* Dimensions: 22" x 16" x 9"
* Weight: 18 lbs

Upgrade From: ChromaJet Pro

Compatible Accessories (Sold Separately):

* ChromaShield Laminator: Protect your prints with a durable laminate finish.
* ChromaClean Kit: Keep your printer running smoothly with this comprehensive cleaning kit.
* ChromaPaper Pro Pack: Experience superior print quality with our premium photo paper.
Category: Printer
Segment: ['Home', 'Office']
Tags: ['Printing']

""",
        """
ChromaShield Laminator

Price: $149.00

Protect your valuable prints from fading, scratches, and moisture with the ChromaShield Laminator. Specifically designed to complement the ChromaJet Pro and ChromaJet Pro XL printers, this laminator ensures your photos and documents maintain their vibrant colors and sharp details for years to come. Easy to use and built to last, the ChromaShield is an essential addition to any home or professional printing setup.

Manufacturer: Imaging Solutions Inc.

Features:

*   Hot and Cold Lamination: Offers versatility for various document types and thicknesses.
*   Jam Release Lever: Prevents document damage in case of misfeeds.
*   Rapid Warm-Up Time: Ready to laminate in just 3 minutes.
*   Compact Design: Fits easily on any desk or workstation.
*   User-Friendly Controls: Simple operation for all skill levels.

Specifications:

*   Maximum Laminating Width: 12 inches
*   Pouch Thickness: 3-5 mil
*   Warm-Up Time: 3 minutes
*   Laminating Speed: 10 inches per minute
*   Dimensions: 16 x 6 x 4 inches
*   Power Consumption: 200W
*   Voltage: 120V AC, 60Hz
*   Weight: 5 lbs
*   Compatible With: ChromaJet Pro, ChromaJet Pro XL Printers
Tags: ['Printing']
""",
        """
 ChromaShield Laminator

Price: $149.00

Protect your valuable prints from fading, scratches, and moisture with the ChromaShield Laminator. Specifically designed to complement the ChromaJet Pro and ChromaJet Pro XL printers, this laminator ensures your photos and documents maintain their vibrant colors and sharp details for years to come. Easy to use and built to last, the ChromaShield is an essential addition to any home or professional printing setup.

Manufacturer: Imaging Solutions Inc.

Features:

*   Hot and Cold Lamination: Offers versatility for various document types and thicknesses.
*   Jam Release Lever: Prevents document damage in case of misfeeds.
*   Rapid Warm-Up Time: Ready to laminate in just 3 minutes.
*   Compact Design: Fits easily on any desk or workstation.
*   User-Friendly Controls: Simple operation for all skill levels.

Specifications:

*   Maximum Laminating Width: 12 inches
*   Pouch Thickness: 3-5 mil
*   Warm-Up Time: 3 minutes
*   Laminating Speed: 10 inches per minute
*   Dimensions: 16 x 6 x 4 inches
*   Power Consumption: 200W
*   Voltage: 120V AC, 60Hz
*   Weight: 5 lbs
*   Compatible With: ChromaJet Pro, ChromaJet Pro XL Printers
Tags: ['Printing']

""",
        """
ChromaClean Kit

Price: $29.99

Manufacturer: Chroma Solutions

Description: Maintain peak performance and extend the life of your ChromaJet Pro or ChromaJet Pro XL printer with the ChromaClean Kit. This comprehensive cleaning solution ensures your printer operates smoothly, producing consistently high-quality prints. Prevent clogs, remove residue, and optimize ink flow with this essential maintenance kit.

Features:

*   Optimized cleaning solution for ChromaJet Pro and ChromaJet Pro XL printers
*   Prevents ink clogs and buildup
*   Removes dried ink and debris
*   Maintains optimal print quality
*   Easy-to-use cleaning process

Specifications:

*   Kit Contents: 1 bottle (4 oz) cleaning solution, 10 cleaning swabs, detailed instructions
*   Solution Composition: Non-toxic, non-corrosive cleaning agents
*   Shelf Life: 2 years
*   Compatibility: ChromaJet Pro, ChromaJet Pro XL
Tags: ['Printing']
""",
        """
 ChromaClean Kit

Price: $29.99

Manufacturer: Chroma Solutions

Description: Maintain peak performance and extend the life of your ChromaJet Pro or ChromaJet Pro XL printer with the ChromaClean Kit. This comprehensive cleaning solution ensures your printer operates smoothly, producing consistently high-quality prints. Prevent clogs, remove residue, and optimize ink flow with this essential maintenance kit.

Features:

*   Optimized cleaning solution for ChromaJet Pro and ChromaJet Pro XL printers
*   Prevents ink clogs and buildup
*   Removes dried ink and debris
*   Maintains optimal print quality
*   Easy-to-use cleaning process

Specifications:

*   Kit Contents: 1 bottle (4 oz) cleaning solution, 10 cleaning swabs, detailed instructions
*   Solution Composition: Non-toxic, non-corrosive cleaning agents
*   Shelf Life: 2 years
*   Compatibility: ChromaJet Pro, ChromaJet Pro XL
Tags: ['Printing']

""",
        """
ChromaPaper Pro Pack

Price: $49.99

Description: Achieve exceptional print quality with the ChromaPaper Pro Pack, the perfect companion for your high-performance printer. This premium paper is specifically designed to deliver vibrant, long-lasting prints with exceptional color accuracy and sharpness. Ideal for professional photographers, graphic designers, and anyone who demands the best possible results from their inkjet printer.

Features:

*   Optimized for inkjet printers
*   Delivers exceptional color vibrancy
*   Fade-resistant for long-lasting prints
*   Acid-free archival quality
*   Smooth, matte finish minimizes glare
*   Quick-drying surface prevents smudging

Specifications:

*   Manufacturer: Chroma Imaging
*   Paper Size: 8.5 x 11 inches (Letter)
*   Paper Weight: 68 lb (250 gsm)
*   Sheets per Pack: 100
*   Finish: Matte
*   Brightness: 98 ISO
*   Opacity: 97%
*   Compatibility: Inkjet printers
*   Accessory Of: ChromaJet Pro XL Printer
Tags: ['Printing']

""",
        """
 ChromaPaper Pro Pack

Price: $49.99

Description: Achieve exceptional print quality with the ChromaPaper Pro Pack, the perfect companion for your high-performance printer. This premium paper is specifically designed to deliver vibrant, long-lasting prints with exceptional color accuracy and sharpness. Ideal for professional photographers, graphic designers, and anyone who demands the best possible results from their inkjet printer.

Features:

*   Optimized for inkjet printers
*   Delivers exceptional color vibrancy
*   Fade-resistant for long-lasting prints
*   Acid-free archival quality
*   Smooth, matte finish minimizes glare
*   Quick-drying surface prevents smudging

Specifications:

*   Manufacturer: Chroma Imaging
*   Paper Size: 8.5 x 11 inches (Letter)
*   Paper Weight: 68 lb (250 gsm)
*   Sheets per Pack: 100
*   Finish: Matte
*   Brightness: 98 ISO
*   Opacity: 97%
*   Compatibility: Inkjet printers
*   Accessory Of: ChromaJet Pro XL Printer
Tags: ['Printing']

""",
        """
ChromaJet Starter Bundle

Price: $528.00

Manufacturer: ChromaCorp

Description: Get started with everything you need for vibrant, professional-quality prints with the ChromaJet Starter Bundle. This bundle includes the flagship ChromaJet Pro printer and the essential ChromaClean Kit, ensuring pristine prints from day one.

Features:

*   Complete Starter Package: Includes printer and cleaning kit for immediate use.
*   Professional-Grade Quality: Produce high-resolution prints with exceptional color accuracy.
*   User-Friendly Interface: Intuitive controls for easy operation, even for beginners.
*   Versatile Media Handling: Supports a wide range of paper types and sizes.
*   Durable Construction: Built to last, ensuring years of reliable performance.

Specifications (ChromaJet Pro):

*   Print Resolution: Up to 4800 x 1200 dpi
*   Print Speed: 15 ppm (black), 10 ppm (color)
*   Paper Sizes: Letter, Legal, 4x6, 5x7, A4, A5
*   Connectivity: USB 2.0, Wi-Fi
*   Ink Cartridges: 4 individual ink cartridges (Cyan, Magenta, Yellow, Black)
*   Dimensions: 17.5 x 13.0 x 6.0 inches
*   Weight: 12 lbs

Specifications (ChromaClean Kit):

*   Includes: Printhead cleaning solution (100ml), lint-free cloths (10), cleaning swabs (10)
*   Purpose: Effectively removes ink buildup and debris from printheads.
Tags: ['Printing']

""",
        """
 ChromaJet Starter Bundle

Price: $528.00

Manufacturer: ChromaCorp

Description: Get started with everything you need for vibrant, professional-quality prints with the ChromaJet Starter Bundle. This bundle includes the flagship ChromaJet Pro printer and the essential ChromaClean Kit, ensuring pristine prints from day one.

Features:

*   Complete Starter Package: Includes printer and cleaning kit for immediate use.
*   Professional-Grade Quality: Produce high-resolution prints with exceptional color accuracy.
*   User-Friendly Interface: Intuitive controls for easy operation, even for beginners.
*   Versatile Media Handling: Supports a wide range of paper types and sizes.
*   Durable Construction: Built to last, ensuring years of reliable performance.

Specifications (ChromaJet Pro):

*   Print Resolution: Up to 4800 x 1200 dpi
*   Print Speed: 15 ppm (black), 10 ppm (color)
*   Paper Sizes: Letter, Legal, 4x6, 5x7, A4, A5
*   Connectivity: USB 2.0, Wi-Fi
*   Ink Cartridges: 4 individual ink cartridges (Cyan, Magenta, Yellow, Black)
*   Dimensions: 17.5 x 13.0 x 6.0 inches
*   Weight: 12 lbs

Specifications (ChromaClean Kit):

*   Includes: Printhead cleaning solution (100ml), lint-free cloths (10), cleaning swabs (10)
*   Purpose: Effectively removes ink buildup and debris from printheads.
Tags: ['Printing']

""",
        """
**Product Summary: ChromaJet Pro XL Studio Bundle**

Elevate your print production with the ChromaJet Pro XL Studio Bundle, designed for professionals seeking unparalleled quality and efficiency. This comprehensive bundle combines the ChromaJet Pro XL printer with the ChromaShield Laminator, providing a complete solution for creating stunning prints that last.

**Price:** $928.00

**Manufacturer:** ChromaTech Solutions

**Features:**

*   Professional-Grade Printing: Achieve vibrant colors, sharp details, and exceptional clarity on a wide range of media.
*   High-Speed Output: Increase productivity with rapid print speeds.
*   Wide Format Capabilities: Supports media up to 13 inches wide, perfect for posters, banners, and large-scale projects.
*   Durable Lamination: Protect your prints from scratches, fading, and moisture with the ChromaShield Laminator.
*   User-Friendly Interface: Intuitive controls and software for easy operation and workflow integration.
*   Versatile Media Handling: Compatible with various paper types, photo papers, and specialty media.
*   Space-Saving Design: Compact footprint optimizes workspace utilization.
*   Includes: ChromaJet Pro XL Printer and ChromaShield Laminator

**Specifications:**

*   **ChromaJet Pro XL Printer:**
    *   Print Resolution: Up to 4800 x 1200 dpi
    *   Print Speed: Up to 15 ppm (black), 10 ppm (color)
    *   Maximum Print Size: 13 x 19 inches
    *   Ink Type: Pigment-based inks for archival quality
    *   Connectivity: USB 2.0, Wi-Fi
    *   Paper Capacity: 100 sheets (plain paper)
    *   Dimensions: 22 x 13 x 9 inches
    *   Weight: 18 lbs
*   **ChromaShield Laminator:**
    *   Laminating Width: Up to 13 inches
    *   Laminating Speed: Up to 12 inches per minute
    *   Film Thickness: 3 mil to 5 mil
    *   Temperature Settings: Adjustable for different film types
    *   Dimensions: 19 x 7 x 4 inches
    *   Weight: 7 lbs

Achieve professional-grade results with the ChromaJet Pro XL Studio Bundle. Order yours today and experience the difference!
Tags: ['Printing']

""",
        """
 **Product Summary: ChromaJet Pro XL Studio Bundle**

Elevate your print production with the ChromaJet Pro XL Studio Bundle, designed for professionals seeking unparalleled quality and efficiency. This comprehensive bundle combines the ChromaJet Pro XL printer with the ChromaShield Laminator, providing a complete solution for creating stunning prints that last.

**Price:** $928.00

**Manufacturer:** ChromaTech Solutions

**Features:**

*   Professional-Grade Printing: Achieve vibrant colors, sharp details, and exceptional clarity on a wide range of media.
*   High-Speed Output: Increase productivity with rapid print speeds.
*   Wide Format Capabilities: Supports media up to 13 inches wide, perfect for posters, banners, and large-scale projects.
*   Durable Lamination: Protect your prints from scratches, fading, and moisture with the ChromaShield Laminator.
*   User-Friendly Interface: Intuitive controls and software for easy operation and workflow integration.
*   Versatile Media Handling: Compatible with various paper types, photo papers, and specialty media.
*   Space-Saving Design: Compact footprint optimizes workspace utilization.
*   Includes: ChromaJet Pro XL Printer and ChromaShield Laminator

**Specifications:**

*   **ChromaJet Pro XL Printer:**
    *   Print Resolution: Up to 4800 x 1200 dpi
    *   Print Speed: Up to 15 ppm (black), 10 ppm (color)
    *   Maximum Print Size: 13 x 19 inches
    *   Ink Type: Pigment-based inks for archival quality
    *   Connectivity: USB 2.0, Wi-Fi
    *   Paper Capacity: 100 sheets (plain paper)
    *   Dimensions: 22 x 13 x 9 inches
    *   Weight: 18 lbs
*   **ChromaShield Laminator:**
    *   Laminating Width: Up to 13 inches
    *   Laminating Speed: Up to 12 inches per minute
    *   Film Thickness: 3 mil to 5 mil
    *   Temperature Settings: Adjustable for different film types
    *   Dimensions: 19 x 7 x 4 inches
    *   Weight: 7 lbs

Achieve professional-grade results with the ChromaJet Pro XL Studio Bundle. Order yours today and experience the difference!
Tags: ['Printing']

""",
        """
Deal: ChromaJet Pro XL Early Bird

Price: $749.00

Manufacturer: ChromaTech

Description: Don't miss out on this limited-time offer! Get the ChromaJet Pro XL, a high-performance inkjet printer designed for professional-quality photo and document printing, at a special early bird price. Offer ends March 15, 2024.

Features:

High-resolution printing: Up to 5760 x 1440 dpi
Fast print speeds: Up to 15 ppm (black), 10 ppm (color)
Borderless printing: Print photos and documents without borders
Wireless connectivity: Wi-Fi and Ethernet for easy network access
Mobile printing: Print from your smartphone or tablet with ChromaTech Print app
Large paper capacity: 250-sheet paper tray
Automatic duplex printing: Save paper with automatic two-sided printing

Specifications:

Print Technology: Inkjet
Ink Type: ChromaTech Dye-based Inks
Paper Sizes: Letter, Legal, 4x6", 5x7", 8x10"
Connectivity: USB 2.0, Wi-Fi, Ethernet
Operating Systems: Windows, macOS
Dimensions: 17.7" x 14.9" x 8.1"
Weight: 14.3 lbs
Warranty: 1-year limited warranty
Tags: ['Printing']
""",
        """
 ChromaJet Pro XL Early Bird

Price: $749.00

Manufacturer: ChromaTech

Description: Don't miss out on this limited-time offer! Get the ChromaJet Pro XL, a high-performance inkjet printer designed for professional-quality photo and document printing, at a special early bird price. Offer ends March 15, 2024.

Features:

High-resolution printing: Up to 5760 x 1440 dpi
Fast print speeds: Up to 15 ppm (black), 10 ppm (color)
Borderless printing: Print photos and documents without borders
Wireless connectivity: Wi-Fi and Ethernet for easy network access
Mobile printing: Print from your smartphone or tablet with ChromaTech Print app
Large paper capacity: 250-sheet paper tray
Automatic duplex printing: Save paper with automatic two-sided printing

Specifications:

Print Technology: Inkjet
Ink Type: ChromaTech Dye-based Inks
Paper Sizes: Letter, Legal, 4x6", 5x7", 8x10"
Connectivity: USB 2.0, Wi-Fi, Ethernet
Operating Systems: Windows, macOS
Dimensions: 17.7" x 14.9" x 8.1"
Weight: 14.3 lbs
Warranty: 1-year limited warranty
Tags: ['Printing']

""",
    ],
    [
        """,
Product Summary: Aura X5 Smartphone

Experience the future of mobile technology with the Aura X5. Featuring a revolutionary holographic display and an intuitive AI assistant, the Aura X5 redefines what a smartphone can be.

Price: $899.00

Manufacturer: Stellaris Tech

Features:

*   Holographic Display: Experience immersive visuals like never before. Project images and videos into the air for a stunning 3D effect.
*   Integrated AI Assistant: "Aura" learns your habits and anticipates your needs. Manage your schedule, answer questions, and control your smart home devices with ease.
*   Advanced Camera System: Capture breathtaking photos and videos with the 108MP main lens, 48MP ultra-wide lens, and 12MP telephoto lens.
*   Blazing-Fast Processor: Powered by the Octa-Core Snapdragon 8 Gen 2 processor for seamless multitasking and lag-free gaming.
*   5G Connectivity: Experience lightning-fast download and streaming speeds.
*   Long-lasting Battery: 5000mAh battery provides all-day power.

Specifications:

*   Display: 6.8-inch Holographic AMOLED Display (2400 x 1080 resolution)
*   Processor: Qualcomm Snapdragon 8 Gen 2
*   RAM: 12GB
*   Storage: 256GB Internal Storage
*   Operating System: StellarisOS 7 (Based on Android 14)
*   Front Camera: 32MP
*   Rear Camera: 108MP (Main) + 48MP (Ultra-Wide) + 12MP (Telephoto)
*   Battery: 5000mAh
*   Connectivity: 5G, Wi-Fi 6E, Bluetooth 5.3, NFC, USB-C
*   Water Resistance: IP68
*   Dimensions: 165.1 x 75.6 x 8.9 mm
*   Weight: 205g

Accessorize your Aura X5:

*   Aura X5 Case: Protect your investment with a custom-designed case. Available in multiple colors. (Sold Separately)
Category: Mobile Phones
Segment: ['Home']
Tags: ['Phone']

""",
        """
Product Summary: Sonaris Pro Noise-Canceling Headphones

Immerse yourself in pristine audio with the Sonaris Pro, noise-canceling headphones engineered for unparalleled sound quality and comfort. Experience your favorite music like never before with revolutionary spatial audio technology that creates a truly immersive listening experience.

Price: $249.00

Manufacturer: Sonaris Audio

Features:

*   Active Noise Cancellation (ANC): Effectively blocks out distracting ambient noise.
*   Spatial Audio: Creates a three-dimensional soundstage for an immersive listening experience.
*   Crystal-Clear Audio: High-fidelity drivers deliver rich, detailed sound.
*   Comfortable Design: Ergonomic earcups and adjustable headband for extended wear.
*   Bluetooth 5.3: Provides a stable and reliable wireless connection.
*   Built-in Microphone: For clear calls and voice assistant access.
*   Long Battery Life: Up to 30 hours of playtime on a single charge (20 hours with ANC).
*   Quick Charge: 10 minutes of charging provides up to 5 hours of playtime.
*   Foldable Design: For easy storage and portability.
*   Intuitive Controls: Easy-to-use controls for volume, playback, and calls.

Specifications:

*   Driver Size: 40mm
*   Impedance: 32 ohms
*   Frequency Response: 20Hz - 20kHz
*   Bluetooth Version: 5.3
*   Bluetooth Range: Up to 33 feet (10 meters)
*   Battery Capacity: 600mAh
*   Charging Time: 2 hours
*   Weight: 0.6 lbs (272 grams)
*   Color: Midnight Black
*   Included Accessories: USB-C charging cable, carrying case, user manual

Enhance your listening experience with the optional Sonaris Pro Charging Stand (sold separately).

Category: Headphones
Segment: ['Home']
Tags: ['Phone']

""",
        """
Product Summary: Aura X5 Pro - Capture Brilliance, Last Longer

Price: $1299.00

Manufacturer: Aura Mobile

Description: Experience the pinnacle of mobile technology with the Aura X5 Pro. Building upon the acclaimed Aura X5, this enhanced model boasts a significantly improved camera system and extended battery life, ensuring you never miss a moment. Capture stunning photos and videos, and stay connected all day long.

Features:

*   Advanced Camera System: Featuring a 50MP primary sensor with OIS, a 12MP ultrawide lens, and a 10MP telephoto lens with 3x optical zoom, the Aura X5 Pro captures professional-quality images in any lighting conditions.
*   Extended Battery Life: The 5000mAh battery delivers all-day power, allowing you to browse, stream, and create without worrying about running out of charge. Supports 65W fast charging.
*   Brilliant Display: Immerse yourself in vibrant colors and sharp details on the 6.7-inch AMOLED display with a 120Hz refresh rate.
*   Powerful Performance: Powered by the Snapdragon 8 Gen 2 processor, the Aura X5 Pro delivers lightning-fast performance for gaming, multitasking, and demanding applications.
*   Enhanced Security: Features an in-display fingerprint sensor and advanced facial recognition for secure and convenient unlocking.
*   Durable Design: Crafted with a premium aluminum frame and Gorilla Glass Victus+ on both the front and back for exceptional durability.

Specifications:

*   Display: 6.7-inch AMOLED, 120Hz refresh rate
*   Processor: Snapdragon 8 Gen 2
*   RAM: 12GB
*   Storage: 256GB
*   Rear Camera: 50MP (OIS) + 12MP (Ultrawide) + 10MP (Telephoto, 3x Optical Zoom)
*   Front Camera: 32MP
*   Battery: 5000mAh, 65W Fast Charging
*   Operating System: Aura UI 5.0 (Based on Android 13)
*   Connectivity: 5G, Wi-Fi 6E, Bluetooth 5.3, NFC, USB-C
*   Water Resistance: IP68
*   Dimensions: 163.5 x 75.5 x 8.2 mm
*   Weight: 205g

Upgrade Of: Aura X5

Compatible Accessories: Aura X5 Case
Category: Mobile Phones
Segment: ['Home']
Tags: ['Phone']

""",
        """
Sonaris Max: Over-Ear Headphones - Experience Superior Sound

Price: $349.00

Manufacturer: Sonaris Audio

Description: Immerse yourself in rich, detailed audio with the Sonaris Max over-ear headphones. Building upon the success of the Sonaris Pro, the Sonaris Max delivers an enhanced listening experience with superior sound quality, unparalleled comfort, and premium materials. Enjoy hours of listening pleasure with plush earcups and a lightweight design.

Features:

*   High-fidelity audio drivers for crystal-clear sound
*   Active Noise Cancellation (ANC) to block out distractions
*   Bluetooth 5.3 for a stable and reliable wireless connection
*   Up to 30 hours of battery life on a single charge (25 hours with ANC)
*   Built-in microphone for clear calls
*   Intuitive on-ear controls for volume, playback, and calls
*   Durable construction with premium materials
*   Foldable design for easy storage and portability

Specifications:

*   Driver Size: 45mm
*   Frequency Response: 20Hz - 20kHz
*   Impedance: 32 Ohms
*   Sensitivity: 105dB +/- 3dB
*   Bluetooth Version: 5.3
*   Wireless Range: Up to 33 feet (10 meters)
*   Battery Life: Up to 30 hours (25 hours with ANC)
*   Charging Time: 2 hours
*   Weight: 9.9 ounces
*   Color: Midnight Black

Accessories (Sold Separately): Enhance your Sonaris Max experience with the Sonaris Pro Charging Stand and Headphone Cleaning Kit.
Category: Headphones
Segment: ['Home']
Tags: ['Phone']
""",
        """
 Sonaris Max: Over-Ear Headphones - Experience Superior Sound

Price: $349.00

Manufacturer: Sonaris Audio

Description: Immerse yourself in rich, detailed audio with the Sonaris Max over-ear headphones. Building upon the success of the Sonaris Pro, the Sonaris Max delivers an enhanced listening experience with superior sound quality, unparalleled comfort, and premium materials. Enjoy hours of listening pleasure with plush earcups and a lightweight design.

Features:

*   High-fidelity audio drivers for crystal-clear sound
*   Active Noise Cancellation (ANC) to block out distractions
*   Bluetooth 5.3 for a stable and reliable wireless connection
*   Up to 30 hours of battery life on a single charge (25 hours with ANC)
*   Built-in microphone for clear calls
*   Intuitive on-ear controls for volume, playback, and calls
*   Durable construction with premium materials
*   Foldable design for easy storage and portability

Specifications:

*   Driver Size: 45mm
*   Frequency Response: 20Hz - 20kHz
*   Impedance: 32 Ohms
*   Sensitivity: 105dB +/- 3dB
*   Bluetooth Version: 5.3
*   Wireless Range: Up to 33 feet (10 meters)
*   Battery Life: Up to 30 hours (25 hours with ANC)
*   Charging Time: 2 hours
*   Weight: 9.9 ounces
*   Color: Midnight Black

Accessories (Sold Separately): Enhance your Sonaris Max experience with the Sonaris Pro Charging Stand and Headphone Cleaning Kit.
Category: Headphones
Segment: ['Home']
Tags: ['Phone']

""",
        """
 Aura X5 Case

Price: $39.00

Manufacturer: Aura Accessories

Description: Protect your Aura X5 or Aura X5 Pro with the Aura X5 Case. This durable and stylish case is designed to safeguard your phone from everyday bumps, scratches, and drops. The sleek design complements the aesthetics of your Aura X5 device while providing a comfortable grip.

Features:

*   Durable construction for impact resistance
*   Slim and lightweight design
*   Precise cutouts for easy access to ports and buttons
*   Raised bezel to protect the screen and camera
*   Stylish finish
*   Easy to install and remove

Specifications:

*   Compatibility: Aura X5, Aura X5 Pro
*   Material: Thermoplastic Polyurethane (TPU)
*   Weight: 1.5 ounces
*   Dimensions: 6.2 x 3.0 x 0.4 inches
*   Available Colors: Black, Clear, Blue, Red
Category: Cases
Segment: ['Home']
Tags: ['Phone']

""",
        """
Product Summary: Sonaris Pro Charging Stand

Price: $29.00

Manufacturer: Sonaris Audio

Description: Keep your Sonaris Pro or Sonaris Max headphones charged and displayed in style with the Sonaris Pro Charging Stand. This elegantly designed stand provides a convenient and stable charging solution, ensuring your headphones are always ready when you are. Its sleek profile complements your headphones and any desktop environment.

Features:

*   Dedicated charging contact points for seamless power transfer.
*   Stable base prevents tipping.
*   Sleek, minimalist design.
*   Protective padding to prevent scratching.
*   Keeps headphones organized and ready for use.

Specifications:

*   Compatibility: Sonaris Pro, Sonaris Max
*   Material: Durable ABS Plastic with soft-touch finish
*   Color: Matte Black
*   Dimensions: 4 inches (width) x 4 inches (depth) x 2.5 inches (height)
*   Weight: 6 ounces
*   Charging Cable: USB-C (cable included)
*   Input: 5V/2A
*   Output: 5V/1A
Category: Accessories
Segment: ['Home']
Tags: ['Phone']
""",
        """
Product Summary: Quantum Shield Screen Protector

Protect your Aura X5 or Aura X5 Pro with the Quantum Shield, an ultra-strong screen protector designed for maximum durability and clarity. Say goodbye to scratches, cracks, and smudges and enjoy peace of mind knowing your screen is safeguarded.

Price: $19

Manufacturer: Tech Armor

Features:

*   Ultra-clear transparency for optimal viewing experience
*   Scratch-resistant coating protects against daily wear and tear
*   Bubble-free installation for a seamless fit
*   Oleophobic coating repels fingerprints and smudges
*   Precise cutouts for sensors and front-facing camera
*   Maintains touchscreen responsiveness
*   Easy to clean

Specifications:

*   Compatibility: Aura X5, Aura X5 Pro
*   Material: Tempered Glass
*   Thickness: 0.33mm
*   Hardness: 9H
*   Package Includes: One (1) Quantum Shield screen protector, cleaning cloth, dust removal stickers, installation guide.
Category: Screen Guards
Segment: ['Home']
Tags: ['Phone']

""",
        """
**Product Summary: Audio Lover's Bundle**

Immerse yourself in a world of unparalleled audio fidelity with the Audio Lover's Bundle. This all-in-one package includes everything you need for an exceptional listening experience, from premium headphones to convenient charging and meticulous care.

Price: $550

Manufacturer: Sonaris Audio

Bundle Includes:

*   Sonaris Pro Headphones: Experience crystal-clear highs, rich mids, and deep bass with these over-ear headphones, expertly engineered for audiophiles.
*   Sonaris Pro Charging Stand: Keep your Sonaris Pro headphones charged and ready to go with this sleek and stylish charging stand. Doubles as a display stand.
*   Headphone Cleaning Kit: Maintain the pristine condition of your headphones with this specially formulated cleaning kit, designed to remove dust, fingerprints, and debris without damaging delicate components.

Sonaris Pro Headphones Specifications:

*   Driver Size: 45mm Dynamic Drivers
*   Frequency Response: 10Hz - 40kHz
*   Impedance: 32 Ohms
*   Sensitivity: 100dB/mW
*   Bluetooth Version: 5.2
*   Wireless Range: Up to 30 feet
*   Battery Life: Up to 30 hours (single charge)
*   Charging Time: 2 hours (headphones), 3 hours (charging stand)
*   Weight: 10.2 ounces
*   Color: Matte Black
*   Material: Aluminum frame, protein leather earcups
Category: Bundles
Segment: ['Home']
Tags: ['Phone']

""",
        """
**Product Summary: Aura X5 Starter Kit**

**Description:** Get started with the revolutionary Aura X5 with this complete starter kit! Includes the Aura X5 device, a protective Aura X5 Case, and the Quantum Shield for enhanced security.

**Price:** $920.00

**Manufacturer:** Aura Technologies

**Features:**

*   **Aura X5:** Cutting-edge neural interface technology for enhanced cognitive function.
*   **Aura X5 Case:** Durable, form-fitting case protects your Aura X5 from scratches and impacts. Made from aerospace-grade polymer.
*   **Quantum Shield:** Encrypts all Aura X5 data with quantum-resistant algorithms, ensuring maximum privacy and security.

**Specifications:**

*   **Aura X5:**
    *   Processor: Neural Core 7.0
    *   Memory: 32GB RAM
    *   Storage: 512GB Solid State Drive
    *   Connectivity: Bluetooth 5.3, Wi-Fi 6E
    *   Dimensions: 4.5" x 2.5" x 0.5"
    *   Weight: 4.2 oz
    *   Battery Life: Up to 12 hours
*   **Aura X5 Case:**
    *   Material: High-Density Polymer
    *   Weight: 1.5 oz
    *   Color: Matte Black
*   **Quantum Shield:**
    *   Encryption: Quantum-Resistant AES-256
    *   Activation: One-Time Activation Key
    *   Compatibility: Aura X5
Category: Bundles
Segment: ['Home']
Tags: ['Phone']

""",
        """
**Summer Sale! Get Aura X5 + Quantum Shield at a Discount!**

**Price:** $875.00

**Manufacturer:** Aura Dynamics

**Description:** Don't miss out on this limited-time summer sale! This bundle includes the cutting-edge Aura X5 smartphone and the revolutionary Quantum Shield screen protector. Upgrade your mobile experience with superior performance and unmatched protection.

**Products Included:**
*   Aura X5 Smartphone
*   Quantum Shield Screen Protector

**Aura X5 Smartphone Features:**

*   Display: 6.8-inch Dynamic AMOLED 2X Display with 120Hz refresh rate
*   Processor: Octa-Core Snapdragon 8 Gen 2 Mobile Platform
*   RAM: 12GB
*   Storage: 256GB Internal Storage
*   Camera: 108MP Wide, 12MP Ultrawide, 10MP Telephoto Rear Cameras; 32MP Front Camera
*   Battery: 5000 mAh with 45W Super Fast Charging
*   Operating System: Android 14 with Aura UI 6.0
*   Connectivity: 5G, Wi-Fi 6E, Bluetooth 5.3, NFC
*   Security: In-display Fingerprint Sensor, Facial Recognition
*   Water Resistance: IP68 Water and Dust Resistance

**Quantum Shield Screen Protector Features:**

*   Material: Tempered Glass with 9H Hardness
*   Thickness: 0.33mm
*   Clarity: 99.9% Transparency
*   Features: Scratch Resistant, Shatterproof, Oleophobic Coating (Anti-Fingerprint)
*   Compatibility: Specifically designed for Aura X5, Full adhesive coverage.

**Sale Ends:** July 31, 2024. Act fast!
Category: Deals
Segment: ['Home']
Tags: ['Phone']

""",
    ],
    [
        """
 **Product Summary: TitanDrive X5000 - 5TB Ruggedized External SSD**

Unleash unparalleled speed and durability with the TitanDrive X5000, the ultimate external SSD for professionals and enthusiasts. This ruggedized 5TB drive provides lightning-fast data transfer rates in a compact and robust design, perfect for on-the-go creatives, gamers, and anyone needing reliable and high-performance storage.

**Price:** $299.99

**Manufacturer:** TitanTech Solutions

**Key Features:**

*   **Extreme Speed:** Experience blazing-fast read/write speeds up to 2000 MB/s for rapid file transfers and seamless workflow.
*   **Ruggedized Design:** Built to withstand the elements with IP67 water and dust resistance, as well as shock and vibration protection.
*   **Massive Storage:** 5TB capacity provides ample space for large files, including 4K/8K video, high-resolution photos, and extensive game libraries.
*   **Universal Compatibility:** Works with Windows, macOS, and Android devices via USB 3.2 Gen 2 Type-C port.
*   **Compact and Portable:** Sleek and lightweight design easily fits in your pocket or bag for convenient portability.
*   **DataSafe Vault Compatible:** Enhanced security features when used with the DataSafe Vault.

**Specifications:**

*   **Capacity:** 5TB
*   **Interface:** USB 3.2 Gen 2 Type-C
*   **Read Speed:** Up to 2000 MB/s
*   **Write Speed:** Up to 2000 MB/s
*   **Operating Temperature:** 0°C to 60°C (32°F to 140°F)
*   **Dimensions:** 4.5" x 2.3" x 0.6" (114.3 x 58.4 x 15.2 mm)
*   **Weight:** 4.2 oz (119 g)
*   **Water Resistance:** IP67
*   **Shock Resistance:** Up to 1500G
*   **Included in Box:** TitanDrive X5000 SSD, USB Type-C to Type-C cable, USB Type-C to Type-A adapter, Quick Start Guide
*   **Upgrade From:** TitanDrive X4000Category: External Hard Drive
Segment: ['Home', 'Office']
Tags: ['Storage', 'Data']

""",
        """
 Product Summary: QuantumLeap Flash

Blazing-fast storage in a sleek, retractable design. The QuantumLeap Flash drive offers a massive 1TB of storage space with the latest USB 4.0 technology for unparalleled transfer speeds. Perfect for backing up large files, transferring videos, or expanding the storage of your laptop or desktop.

Price: $49.99

Manufacturer: QuantumLeap

Features:

*   1TB Storage Capacity
*   USB 4.0 Interface
*   Retractable Design: Protects the USB connector when not in use.
*   Plug-and-Play: No drivers required.
*   Durable Aluminum Casing
*   Compact and Portable
*   LED Activity Indicator

Specifications:

*   Capacity: 1TB (1000GB)
*   Interface: USB 4.0 (backward compatible with USB 3.2, USB 3.1, and USB 3.0/2.0)
*   Read Speed: Up to 800 MB/s
*   Write Speed: Up to 600 MB/s
*   Dimensions: 2.5 in x 0.8 in x 0.4 in
*   Weight: 0.8 ounces
*   Operating Temperature: 32°F to 140°F (0°C to 60°C)
*   Storage Temperature: -4°F to 185°F (-20°C to 85°C)
*   Compatibility: Windows, macOS, Linux
*   Material: Aluminum Alloy, ABS Plastic
*   Color: Space Gray

Related Accessory: Universal Adapter Kit (Sold Separately) - Enhance your QuantumLeap Flash experience with our versatile adapter kit.
Category: USB Flash Drive
Segment: ['Home', 'Office']
Tags: ['Storage', 'Data']

""",
        """
 DataSafe Vault

Price: $39.99

Manufacturer: SecureTech Solutions

Description: Keep your sensitive data safe and secure with the DataSafe Vault. This rugged and encrypted hard drive case provides superior protection against unauthorized access, accidental damage, and environmental hazards. Perfect for safeguarding personal files, business documents, or any data you need to keep private.

Features:

*   Military-grade encryption: 256-bit AES hardware encryption ensures your data is unreadable without the correct password.
*   Durable construction: Constructed with a shock-resistant polymer shell and a water-resistant seal to withstand harsh conditions.
*   Tamper-proof design: Features a tamper-evident seal to alert you to any unauthorized access attempts.
*   Password protection: Customizable password with alphanumeric and special character support.
*   LED indicator: Clearly displays drive activity and encryption status.
*   USB 3.0 connectivity: High-speed data transfer for quick and efficient backups.

Specifications:

*   Compatible drive size: 2.5-inch SATA HDD/SSD
*   Interface: USB 3.0 (backward compatible with USB 2.0)
*   Encryption type: 256-bit AES hardware encryption
*   Material: High-impact polymer, rubber gasket
*   Dimensions: 5.0 x 3.2 x 0.75 inches
*   Weight: 4 ounces
*   Operating temperature: 32°F to 140°F (0°C to 60°C)
*   Power supply: USB powered
*   Accessory of: TitanDrive X5000
Tags: ['Storage', 'Data']

""",
        """
 Product Summary: Universal Adapter Kit

Effortlessly connect your devices with the Universal Adapter Kit. This essential kit provides seamless compatibility across various USB standards, ensuring you're always prepared. Specifically designed as an accessory for the QuantumLeap Flash drive, this kit greatly expands compatibility with a wider range of devices.

Price: $19.99

Manufacturer: Generic

Features:

*   USB-A to USB-C Adapter: Connect USB-C devices to standard USB-A ports.
*   USB-C to USB-A Adapter: Connect USB-A devices to USB-C ports.
*   USB-C to Micro-USB Adapter: Connect Micro-USB devices to USB-C ports.
*   Compact and Portable: Easily fits in your bag or pocket.
*   Durable Construction: Built to withstand daily use.

Specifications:

*   Adapter Type: USB-A to USB-C, USB-C to USB-A, USB-C to Micro-USB
*   Connector Material: Nickel-Plated
*   Enclosure Material: ABS Plastic
*   Color: Black
*   Weight: 0.8 oz
*   Accessory Of: QuantumLeap Flash
*   Data Transfer Rate: Up to 5 Gbps (depending on device and port)
Tags: ['Storage', 'Data']

""",
        """
 Product Summary: Secure Erase Tool

Price: $9.99

Description: Ensure your sensitive data is permanently and securely erased with the Secure Erase Tool. This software provides a reliable method for completely wiping hard drives and flash drives, preventing data recovery and protecting your privacy. It's the perfect solution when retiring older drives, selling used devices, or simply maintaining data security.

Manufacturer: DataSafe Solutions

Features:
* Securely overwrites data multiple times to prevent recovery.
* Supports various wiping standards including DoD 5220.22-M, Gutmann, and custom configurations.
* User-friendly interface for easy operation.
* Compatible with HDDs, SSDs, and USB flash drives.
* Generates detailed reports for verification.
* Quick Erase and Full Erase options available.

Specifications:
* Software Type: Data Erasure Utility
* Supported Operating Systems: Windows 10, Windows 8.1, Windows 7
* File System Support: FAT32, NTFS, exFAT
* Disk Interface Support: SATA, IDE, USB
* License: Single-User License
* File Size: 5MB
* Minimum System Requirements: 1 GHz Processor, 512 MB RAM, 10 MB Hard Disk Space
* Accessory Of: TitanDrive X5000, QuantumLeap Flash
Tags: ['Storage', 'Data']

""",
        """
 TitanDrive X5000 Backup Bundle - Safeguard Your Data

Price: $329.99

Manufacturer: TitanDrive

Description: The TitanDrive X5000 Backup Bundle provides a complete solution for protecting your valuable data. This bundle combines the high-performance TitanDrive X5000 external hard drive with our user-friendly Secure Erase Tool, ensuring both reliable backups and secure data disposal when needed.

Features:

*   High-Speed Data Transfer: Experience blazing-fast transfer speeds for quick and efficient backups.
*   Large Storage Capacity: Store all your important documents, photos, videos, and more.
*   Plug-and-Play Simplicity: Easy to set up and use, no technical expertise required.
*   Automatic Backup Software: Schedule automatic backups to ensure your data is always protected.
*   Secure Data Erasure: Permanently and securely erase sensitive data from your old drives.
*   Durable and Reliable: Built to last with high-quality components for long-term reliability.
*   Compact and Portable: Take your data with you wherever you go.

Specifications (TitanDrive X5000):

*   Capacity: 4TB
*   Interface: USB 3.2 Gen 1 (backward compatible with USB 3.0 and USB 2.0)
*   Data Transfer Rate: Up to 5 Gbps
*   Dimensions: 4.5 x 3.2 x 0.8 inches
*   Weight: 8 ounces
*   Operating System Compatibility: Windows 10, Windows 8.1, Windows 7, macOS 10.13 or later
*   Power Supply: USB Powered

Secure Erase Tool:

*   Compatibility: Windows 10, Windows 8.1, Windows 7, macOS 10.13 or later
*   Erase Methods: Supports multiple secure erase standards including DoD 5220.22-M, Gutmann, and more.
Tags: ['Storage', 'Data']

""",
        """
 QuantumLeap Ultimate Kit

Price: $64.99

Manufacturer: QuantumLeap Technologies

Description: Unleash the full potential of your data with the QuantumLeap Ultimate Kit! This comprehensive bundle includes the blazing-fast QuantumLeap Flash drive and the versatile Universal Adapter Kit, providing seamless connectivity across all your devices.

Features:

High-Speed Data Transfer: Experience lightning-fast file transfers with the QuantumLeap Flash drive.
Universal Compatibility: The Universal Adapter Kit ensures you can connect to USB-A, USB-C, and Micro-USB ports.
Durable and Reliable: Built with high-quality components for long-lasting performance.
Portable and Compact: Take your data on the go with the kit's slim and lightweight design.
Plug and Play: No drivers required – simply plug in and start transferring files.

Specifications:

QuantumLeap Flash:
Capacity: 256GB
Interface: USB 3.2 Gen 2
Read Speed: Up to 400 MB/s
Write Speed: Up to 300 MB/s
Material: Aluminum Alloy
Dimensions: 2.5 x 0.8 x 0.4 inches
Weight: 0.5 oz

Universal Adapter Kit:
Connectors: USB-A, USB-C, Micro-USB
Material: Durable Plastic
Cable Length: 6 inches (approximate)
Compatibility: Works with most devices that support USB data transfer.
Includes: USB-C to USB-A adapter, Micro-USB to USB-A adapter.

Tags: ['Storage', 'Data']

""",
        """
 **QuantumLeap Flash Friday Deal - Unleash Blazing Fast Storage!**

**Price:** $39.99

**Manufacturer:** QuantumStorage Technologies

**Description:** Don't miss out on this Flash Friday exclusive! Grab the QuantumLeap Flash drive and experience lightning-fast data transfer speeds. This high-performance USB 3.2 Gen 2 flash drive is perfect for storing and transferring large files, backing up important documents, or running portable applications. This offer is only available for a limited time, so act fast!

**Key Features:**

*   Blazing-Fast Transfer Speeds: Experience up to 400MB/s read and 300MB/s write speeds for rapid file transfers.
*   USB 3.2 Gen 2 Interface: Leverage the latest USB technology for optimal performance.
*   Durable and Compact Design: The sleek, lightweight design makes it easy to take your data on the go.
*   Plug and Play: Simply plug it in and start transferring files instantly, no drivers required.
*   Wide Compatibility: Works with Windows, macOS, and Linux operating systems.
*   Secure Data Storage: Keep your files safe and secure with reliable flash memory.

**Specifications:**

*   Capacity: 256GB
*   Interface: USB 3.2 Gen 2 Type-A
*   Read Speed: Up to 400MB/s
*   Write Speed: Up to 300MB/s
*   Dimensions: 2.5 inches x 0.8 inches x 0.4 inches
*   Weight: 0.5 ounces
*   Operating Temperature: 32°F to 158°F (0°C to 70°C)
*   Storage Temperature: -4°F to 185°F (-20°C to 85°C)
*   Warranty: 1-Year Limited Warranty

**Limited Time Offer: Sale ends March 1, 2024!**
Tags: ['Storage', 'Data']

""",
    ],
]

document_lists = []
for ts_list in text_snippets:
    document_lists.append([Document(page_content=t) for t in ts_list])

In [ ]:
# @title Extract Nodes and Edges
import copy
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_google_vertexai import ChatVertexAI, VertexAIEmbeddings


def print_graph(graph_documents):
    for doc in graph_documents:
        print(doc.source.page_content[:100])
        nodes = copy.deepcopy(doc.nodes)
        for node in nodes:
            if "embedding" in node.properties:
                node.properties["embedding"] = "..."
        print(nodes)
        print(doc.relationships)
        print()


llm = ChatVertexAI(model="gemini-2.0-flash", temperature=0)
llm_transformer = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=["Category", "Segment", "Tag", "Product", "Bundle", "Deal"],
    allowed_relationships=[
        "In_Category",
        "Tagged_With",
        "In_Segment",
        "In_Bundle",
        "Is_Accessory_Of",
        "Is_Upgrade_Of",
        "Has_Deal",
    ],
    node_properties=[
        "name",
        "price",
        "weight",
        "deal_end_date",
        "features",
    ],
)

graph_documents = []
for document_list in document_lists:
    graph_documents.extend(llm_transformer.convert_to_graph_documents(document_list))

# Add embeddings to the graph documents for Product nodes
embedding_service = VertexAIEmbeddings(model_name="text-embedding-004")
for graph_document in graph_documents:
    for node in graph_document.nodes:
        if "features" in node.properties:
            node.properties["embedding"] = embedding_service.embed_query(
                node.properties["features"]
            )

print_graph(graph_documents)

In [ ]:
# @title Post process extracted nodes and edges
# Apply your domain knowledge to clean up and make desired fixes to the
# generated graph in the earlier step.

# set of all valid products
products = set()


def prune_invalid_products():
    for graph_document in graph_documents:
        nodes_to_remove = []
        relationships_to_remove = []
        for node in graph_document.nodes:
            if node.type == "Product" and "features" not in node.properties:
                nodes_to_remove.append(node)
            else:
                products.add(node.id)
        for node in nodes_to_remove:
            graph_document.nodes.remove(node)


def prune_invalid_segments(valid_segments):
    for graph_document in graph_documents:
        nodes_to_remove = []
        for node in graph_document.nodes:
            if node.type == "Segment" and node.id not in valid_segments:
                nodes_to_remove.append(node)
        for node in nodes_to_remove:
            graph_document.nodes.remove(node)


def is_not_a_listed_product(node):
    if node.type == "Product" and node.id not in products:
        return True
    return False


def fix_directions(relation_name, wrong_source_type):
    for graph_document in graph_documents:
        for relationship in graph_document.relationships:
            if relationship.type == relation_name:
                if relationship.source.type == wrong_source_type:
                    source = relationship.source
                    target = relationship.target
                    relationship.source = target
                    relationship.target = source


def prune_dangling_relationships():
    # now remove all dangling relationships
    for graph_document in graph_documents:
        relationships_to_remove = []
        for relationship in graph_document.relationships:
            if is_not_a_listed_product(relationship.source) or is_not_a_listed_product(
                relationship.target
            ):
                relationships_to_remove.append(relationship)
        for relationship in relationships_to_remove:
            graph_document.relationships.remove(relationship)


def prune_unwanted_relationships(relation_name, source, target):
    node_types = set([source, target])
    for graph_document in graph_documents:
        relationships_to_remove = []
        for relationship in graph_document.relationships:
            if (
                relationship.type == relation_name
                and set([relationship.source.type, relationship.target.type])
                == node_types
            ):
                relationships_to_remove.append(relationship)
        for relationship in relationships_to_remove:
            graph_document.relationships.remove(relationship)


prune_invalid_products()
prune_invalid_segments(set(["Home", "Office", "Fitness"]))
prune_unwanted_relationships("IN_CATEGORY", "Bundle", "Category")
prune_unwanted_relationships("IN_CATEGORY", "Deal", "Category")
prune_unwanted_relationships("IN_SEGMENT", "Bundle", "Segment")
prune_unwanted_relationships("IN_SEGMENT", "Deal", "Segment")
prune_dangling_relationships()
fix_directions("HAS_DEAL", "Deal")
fix_directions("IN_BUNDLE", "Bundle")
print_graph(graph_documents)

In [ ]:
# @title Load data to Spanner Graph
# Uncomment the line below, if you want to cleanup from
# previous iterations.
# BeWARE - THIS COULD REMOVE DATA FROM YOUR DATABASE !!!
graph_store.cleanup()
graph_store.add_graph_documents(graph_documents)

### Visualization

In [ ]:
# @title Graph and Schema
%%spanner_graph --project {PROJECT_ID} --instance {INSTANCE} --database {DATABASE}
GRAPH my_graph #Change this to your graph name
MATCH p = ()->()
RETURN TO_JSON(p) AS path_json

## GraphRAG flow using Spanner Graph

In [ ]:
# @title Question Answering Prompt
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts.prompt import PromptTemplate
from langchain_google_vertexai import ChatVertexAI, VertexAIEmbeddings

from IPython.display import Markdown
import textwrap


# Retrieve and generate using the relevant snippets of the blog.
def format_docs(docs):
    print("Context Retrieved: \n")
    for doc in docs:
        print(textwrap.fill(doc.page_content, width=80))
        print("\n")

    context = "\n\n".join(doc.page_content for doc in docs)
    return context


SPANNERGRAPH_QA_TEMPLATE = """
You are a helpful and friendly AI assistant for question answering tasks for an electronics
retail online store.
Create a human readable answer for the for the question.
You should only use the information provided in the context and not use your internal knowledge.
Don't add any information.
Here is an example:

Question: Which funds own assets over 10M?
Context:[name:ABC Fund, name:Star fund]"
Helpful Answer: ABC Fund and Star fund have assets over 10M.

Follow this example when generating answers.
You are given the following information:
- `Question`: the natural language question from the user
- `Graph Schema`: contains the schema of the graph database
- `Graph Query`: A Spanner Graph GQL query equivalent of the question from the user used to extract context from the graph database
- `Context`: The response from the graph database as context. The context has nodes and edges. Use the relationships.
Information:
Question: {question}
Graph Schema: {graph_schema}
Context: {context}

Format your answer to be human readable.
Use the relationships in the context to answer the question.
Only include information that is relevant to a customer.
Helpful Answer:"""

prompt = PromptTemplate(
    template=SPANNERGRAPH_QA_TEMPLATE,
    input_variables=["question", "graph_schema", "context"],
)

llm = ChatVertexAI(model="gemini-2.0-flash", temperature=0)

chain = prompt | llm | StrOutputParser()

In [ ]:
# @title GraphRAG using Vector Search and Graph Expansion
import textwrap
from langchain_google_spanner import SpannerGraphVectorContextRetriever
from langchain_google_vertexai import VertexAIEmbeddings


def use_node_vector_retriever(
    question, graph_store, embedding_service, label_expr, expand_by_hops
):
    retriever = SpannerGraphVectorContextRetriever.from_params(
        graph_store=graph_store,
        embedding_service=embedding_service,
        label_expr=label_expr,
        expand_by_hops=expand_by_hops,
        top_k=1,
        k=10,
    )
    context = format_docs(retriever.invoke(question))
    return context


embedding_service = VertexAIEmbeddings(model_name="text-embedding-004")
question = "Give me recommendations for a beginner drone"
context = use_node_vector_retriever(
    question, graph_store, embedding_service, label_expr="Product", expand_by_hops=1
)

answer = chain.invoke(
    {"question": question, "graph_schema": graph_store.get_schema, "context": context}
)

print("\n\nAnswer:\n")
print(textwrap.fill(answer, width=80))

Context Retrieved: 

[{"kind": "node", "labels": ["Product"], "properties": {"deal_end_date": null,
"features": "Simple Controls: Beginner friendly and intuitive controls, plus
automatic takeoff and landing.\nTough Build: Designed to handle rookie mistakes,
thanks to its robust construction.\nCapture Memories: Record crisp HD photos and
videos from above.\nExtended Fun: Enjoy up to 15 minutes of flight time per
charge.\nWorry-Free Flying: Free Fly mode lets you fly without directional
concerns.", "id": "Skyhawk Zephyr Drone", "name": null, "price": "$129.99",
"weight": "220g (7.8 oz)"}}]


[{"kind": "node", "labels": ["Product"], "properties": {"deal_end_date": null,
"features": "Simple Controls: Beginner friendly and intuitive controls, plus
automatic takeoff and landing.\nTough Build: Designed to handle rookie mistakes,
thanks to its robust construction.\nCapture Memories: Record crisp HD photos and
videos from above.\nExtended Fun: Enjoy up to 15 minutes of flight time per
charge.\n

## Compare with Conventional RAG

In [ ]:
# @title Define table name
TABLE_NAME = "my_table"  # @param {"type":"string"}

In [ ]:
# @title Setup and load data for vector search
from langchain_google_spanner import SpannerVectorStore
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

import uuid


def load_data_for_vector_search(splits):
    embeddings = VertexAIEmbeddings(model_name="text-embedding-004")

    SpannerVectorStore.init_vector_store_table(
        instance_id=INSTANCE,
        database_id=DATABASE,
        table_name=TABLE_NAME,
    )
    db = SpannerVectorStore(
        instance_id=INSTANCE,
        database_id=DATABASE,
        table_name=TABLE_NAME,
        embedding_service=embeddings,
    )
    # Add the chunks to Spanner Vector Store with embeddings
    ids = [str(uuid.uuid4()) for _ in range(len(splits))]
    row_ids = db.add_documents(splits, ids)


# Create splits for documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
splits = text_splitter.split_documents(
    [document for document_list in document_lists for document in document_list]
)

# Initialize table and load data
embeddings = VertexAIEmbeddings(model_name="text-embedding-004")
load_data_for_vector_search(splits)

Waiting for operation to complete...


In [ ]:
# @title Create a conventional RAG chain

from langchain_core.runnables import RunnablePassthrough
from langchain_google_spanner import SpannerVectorStore
import textwrap


# Retrieve and generate using the relevant snippets of the blog.
def format_docs(docs):
    print("Context Retrieved: \n")
    for doc in docs:
        print(textwrap.fill(doc.page_content, width=80))
        print("\n")

    context = "\n\n".join(doc.page_content for doc in docs)
    return context


prompt = PromptTemplate(
    template="""
    You are a friendly digital shopping assistant.
    Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    Question: {question}
    Context: {context}
    Answer:
  """,
    input_variables=["context", "question"],
)

# Create a rag chain
embeddings = VertexAIEmbeddings(model_name="text-embedding-004")

db = SpannerVectorStore(
    instance_id=INSTANCE,
    database_id=DATABASE,
    table_name=TABLE_NAME,
    embedding_service=embeddings,
)
vector_retriever = db.as_retriever(search_kwargs={"k": 3})
rag_chain = (
    {
        "context": vector_retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# @title Run conventional RAG chain
import textwrap

question = "I am looking for an beginner drone. Please give me some recommendations"
resp = rag_chain.invoke(question)
print("\n\nRag Response:\n")
print(textwrap.fill(resp, width=80))

Context Retrieved: 

Product: SkyHawk Zephyr Drone   Price: $129.99   Weight: 220g (7.8 oz)   The
SkyHawk Zephyr is the perfect drone for beginners. It's built for effortless
flying, offering a smooth and enjoyable experience from the moment you unpack
it.   Features:   Simple Controls: Beginner friendly and intuitive controls,
plus automatic takeoff and landing.   Tough Build: Designed to handle rookie
mistakes, thanks to its robust construction.   Capture Memories: Record crisp HD
photos and videos from above.   Extended Fun: Enjoy up to 15 minutes of flight
time per charge.   Worry-Free Flying: Free Fly mode lets you fly without
directional concerns.   Take your first flight with the SkyHawk Zephyr and
discover the joy of aerial views!    Category: Drone   Segment: ['Home']   Tags:
['Photography', 'Videography']


Bundle:SkyHawk Zephyr Starter Package   Price: $129.99   Everything you need to
begin your drone journey:   This package includes the essentials to get you
flying and capt

## Clean up the graph

> USE IT WITH CAUTION!

Clean up all the nodes/edges in your graph and remove your graph definition.

In [ ]:
graph_store.cleanup()